In [1]:
import sys; sys.path.append('..'); sys.path.append('../..')
from common.postgresql import PostgresConnector
import pandas as pd
import re

db = PostgresConnector()
read_sql = db.read_sql
sql = db.execute_sql

# Leitura de Dados
Lendo gestores da CVM. A query foi construída para facilitar a inserção de novas tabelas via UNION.

In [2]:
query = """
SELECT DISTINCT gestor, 'cvm.cadastro' as tabela 
FROM cvm.cadastro 
WHERE dt_fim IS NULL AND pf_pj_gestor = 'PJ'
"""
# Exemplo de como adicionar mais tabelas:
# query += """ UNION SELECT DISTINCT gestor, 'outra.tabela' as tabela FROM outra.tabela ... """

df_gestores = read_sql(query)
print(f"Total de gestores encontrados: {len(df_gestores)}")
df_gestores.head()

Total de gestores encontrados: 1463


,gestor,tabela
0,10B GESTORA DE RECURSOS LTDA.,cvm.cadastro
1,23S CAPITAL LTDA.,cvm.cadastro
2,2B CAPITAL S.A.,cvm.cadastro
3,3 ILHAS INVESTIMENTOS - GESTÃO DE RECURSOS LTDA.,cvm.cadastro
4,3G CAPITAL GESTORA DE RECURSOS LTDA,cvm.cadastro


In [ ]:
import xlwings as xw
xw.view(df_gestores)

: 

# Definição de Padrões (De-Para)
Lista de tuplas `(Grupo, Regex Pattern)`. 
**Regra para nomes curtos (< 4 chars):** São tratados como palavras inteiras (match exato ou cercado por espaços) para evitar falsos positivos (ex: 'BB' não deve pegar 'BBA').

In [ ]:
# Lista de padrões. A ordem importa (primeiro match vence).
patterns = [
    ('Itaú', r'Ita(ú|u)'),
    ('Intrag', r'Intrag'), # Ajustado para diferenciar do Itaú, ou se for para capturar 'Intrag' com regra específica
    ('BB', r'BB'),
    ('Bradesco', r'Bradesco'),
    ('Caixa', r'Caixa'),
    ('Safra', r'Safra'),
    ('Santander', r'Santander'),
    ('BTG', r'BTG'),
    ('XP', r'XP'),
    ('Vinci', r'Vinci'),
    ('Credit Suisse', r'Credit Suisse'),
    ('Credit Suisse', r'CSHG'),
    ('BofA', r'Bank of America'),
    ('Citibank', r'Citibank'),
    ('Citibank', r'Citi'),
    ('Daycoval', r'Daycoval'),
    ('Bocom', r'Bocom'),
    ('Opportunity', r'Opportunity'),
    ('Pátria', r'P(á|a)tria'),
    ('Kinea', r'Kinea'),
    ('ARX', r'ARX')
]

# Nota: O usuário pediu especificamente: [('Itaú', r'Ita(ú|u)'), ('Intrag', r'Ita(ú|u)')]
# Se o objetivo for mapear 'Ita(ú|u)' para Intrag também, isso geraria conflito se estiverem na mesma lista.
# Mantendo a lógica de pattern matching robusta e prioridade de lista.

In [ ]:
10B GESTORA DE RECURSOS LTDA.	('10b', r'10B')
23S CAPITAL LTDA.	('23s', r'23S')
2B CAPITAL S.A.	('2b', r'2B')
3 ILHAS INVESTIMENTOS - GESTÃO DE RECURSOS LTDA.	('3', r'3 ILHAS')
3G CAPITAL GESTORA DE RECURSOS LTDA	('3g', r'3G')
3J GESTORA DE RECURSOS LTDA. - ME	('3j', r'3J')
3Q ASSET LTDA	('3q', r'3Q')
3R GESTORA DE RECURSOS LTDA	('3r', r'3R')
3V CAPITAL GESTÃO DE RECURSOS LTDA.	('3v', r'3V')
4I CAPITAL LTDA	('4i', r'4I')
4K INVESTIMENTOS LTDA	('4k', r'4K')
4UM DTVM S.A.	('4um', r'4UM')
4UM GESTÃO DE RECURSOS LTDA.	('4um', r'4UM')
A1 INVESTIMENTOS GESTAO DE RECURSOS LTDA	('A1', r'A1 ')
A10 INVESTIMENTOS LTDA.	('A10', r'A10 ')
A3 PERFORMANCE GESTAO DE RECURSOS LTDA	('A3', r'A3 ')
A4X GESTORA DE RECURSOS LTDA	('A4x', r'A4X')
A5 GESTÃO DE INVESTIMENTOS LTDA	('A5', r'A5')
AAA GESTAO DE RECURSOS S/C LTDA	('AAA Gestão', r'AAA ')
AAM GESTORA DE RECURSOS DE TERCEIROS LTDA.	('AAM Gestora', r'AAM')
ABACO GESTAO DE RECURSOS LTDA	('Abaco', r'ABACO')
ABC AGENTES DE INVESTIMENTOS LTDA.	('ABC Capital', r'ABC')
ABC CAPITAL - GESTAO DE INVESTIMENTOS LTDA.	('ABC Capital', r'ABC')
ABC CAPITAL MFO GESTÃO DE ATIVOS LTDA.	('ABC Capital', r'ABC')
ABN AMRO ASSET MANAGEMENT S.A.	('ABN Amro', r'ABN')
ABRADINVEST GESTAO DE RECURSOS FINANCEIROS LTDA	('Abradinvest', r'ABRADINVEST')
ABRDN BRASIL INVESTIMENTOS LTDA.	('Abrdn', r'ABRDN')
ABSOLUTE CRÉDITO GESTÃO DE INVESTIMENTOS LTDA.	('Absolute', r'ABSOLUTE')
ABSOLUTE DHAMA GESTÃO DE INVESTIMENTOS LTDA	('Absolute', r'ABSOLUTE')
ABSOLUTE GESTAO DE INVESTIMENTOS LTDA	('Absolute', r'ABSOLUTE')
ABSOLUTO PARTNERS GESTÃO DE RECURSOS LTDA.	('Absoluto Partners', r'ABSOLUTO')
ACE CAPITAL GESTORA DE RECURSOS LTDA.	('Ace Capital', r'ACE ')
ACE CAPITAL SAIRES GESTORA DE RECURSOS LTDA.	('Ace Capital', r'ACE ')
ACRUX ADMINISTRACAO DE RECURSOS LTDA	('Acrux', r'ACRUX')
ACTUM CAPITAL GESTÃO DE RECURSOS LTDA.	('Actum Capital', r'ACTUM')
ACURA GESTORA DE RECURSOS LTDA	('Acura Gestora', r'ACURA')
ACUTO CAPITAL LTDA.	('Acuto Capital', r'ACUTO')
ADA CAPITAL GESTÃO DE RECURSOS LTDA.	('Ada Capital', r'ADA ')
ADAMCAPITAL GESTÃO DE RECURSOS LTDA.	('Adam Capital', r'ADAMCAPITAL')
ADLER INVESTIMENTOS LTDA.	('Adler Investimentos', r'ADLER')
AF INVEST ADMINISTRAÇÃO DE RECURSOS LTDA	('AF Invest', r'^AF ')
ÁFIRA GESTÃO DE RECURSOS LTDA.	('Áfira', r'ÁFIRA')
AGA GESTORA DE RECURSOS LTDA.	('Aga Gestão', r'AGA')
AGBI ATIVOS REAIS LTDA	('Agbi Ativos', r'AGBI ')
AGG INVESTIMENTOS LTDA.	('Agg Investimentos', r'AGG ')
AGILE ASSET GESTÃO DE INVESTIMENTOS LTDA	('Agile Asset', r'AGILE ASSET')
AGORA CTVM S.A.	('Bradesco', r'AGORA')
ÁGORA GESTAO DE RECURSOS LTDA.	('Bradesco', r'ÁGORA')
AGUILA CAPITAL ADMINISTRAÇÃO E GESTÃO DE CAPITAIS LTDA.	('Aguila', r'AGUILA ')
AJS ASSET MANAGEMENT LTDA.	('AJS Asset', r'AJS ')
AKAD SEGUROS S.A.	('Akad', r'AKAD ')
ALAMEDA CAPITAL GESTORA DE INVESTIMENTOS LTDA.	('Alameda Capital', r'ALAMEDA CAPITAL')
ALAMOS ADMINISTRADORA DE RECURSOS LTDA	('Alamos', r'ALAMOS')
ALASKA INVESTIMENTOS LTDA.	('Alaska Investimentos', r'ALASKA')
ALERCE GESTÃO DE RECURSOS DE TERCEIROS LTDA.	('Alerce Gestão', r'ALERCE')
ALFA PREVIDÊNCIA E VIDA S/A	('Alfa Prev', r'ALFA')
ALGARVE CAPITAL GESTÃO DE RECURSOS LTDA.	('Algarve Capital', r'ALGARVE')
ALGARVE GESTAO DE INVESTIMENTOS LTDA	('Algarve Capital', r'ALGARVE')
ALIANZA GESTÃO DE RECURSOS LTDA	('Alianza', r'ALIANZA')
ALIS INVESTIMENTOS LTDA.	('Alis', r'ALIS')
ALOCC GESTÃO PATRIMONIAL LTDA.	('Alocc', r'ALOCC')
ALP CAPITAL GESTAO DE ATIVOS LTDA	('Alp Capital', r'ALP CAPITAL')
ALPHA CAPITAL PARTNERS GESTAO DE INVESTIMENTOS S.A	('Alpha Capital', r'ALPHA CAPITAL')
ALPHA WAVE CAPITAL LTDA.	('Alpha Wave Capital', r'ALPHA WAVE')
ALPHABOT INVESTIMENTOS S.A.	('Alphabot Investimentos', r'ALPHABOT')
ALPHAKEY CAPITAL MANAGEMENT INVESTIMENTOS LTDA	('Alphakey Capital', r'ALPHAKEY CAPITAL')
ALPHATREE CAPITAL GESTÃO DE RECURSOS LTDA.	('Alphatree Capital', r'ALPHATREE')
ALTERA CAPITAL GESTORA E CONSULTORIA DE INVESTIMENTOS LTDA	('Altera Capital', r'ALTERA CAPITAL')
ALTINVEST GESTÃO E ADMINISTRAÇÃO DE RECURSOS DE TERCEIROS LTDA.	('Altinvest', r'ALTINVEST')
ALTO CAPITAL GESTORA DE RECURSOS LTDA.	('Alto Capital', r'ALTO CAPITAL')
AMAGGI CAPITAL LTDA.	('Amaggi Capital', r'AMAGGI CAPITAL')
AMAZONIA INVESTIMENTOS LTDA	('Amazonia Investimentos', r'AMAZONIA')
AMERICA P.E. ADMINISTRAÇÃO DE RECURSOS LTDA	('America P.E. Administração', r'AMERICA P.E.')
AMERICAS ASSET CONSULTORIA E GESTÃO DE INVESTIMENTOS LTDA.	('Americas Asset', r'AMERICAS ASSET')
AMPLIO CAPITAL LTDA	('Amplio Capital', r'AMPLIO CAPITAL')
AMS CAPITAL LTDA	('AMS Capital', r'AMS CAPITAL')
AMW ASSET MANAGEMENT LTDA	('AMW Asset', r'AMW ASSET')
ANDBANK DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Andbank', r'ANDBANK')
ANDBANK GESTAO DE PATRIMONIO FINANCEIRO LTDA	('Andbank', r'ANDBANK')
ANGÁ ADMINISTRAÇÃO DE RECURSOS LTDA.	('Angá', r'ANGÁ')
ANGÁ INVESTIMENTOS LTDA	('Angá', r'ANGÁ')
ANGRA PARTNERS GESTÃO DE RECURSOS LTDA.	('Angra Partners', r'ANGRA PARTNERS')
ANTARES ADMINISTRADORA DE RECURSOS S/C LTDA	('Antares', r'ANTARES')
ANTARES ASSET MANAGEMENT GESTÃO DE ATIVOS LTDA.	('Antares', r'ANTARES')
ANTHARUS GESTORA DE RECURSOS LTDA.	('Antharus', r'ANTHARUS')
APEX ADMINISTRADORA DE RECURSOS LTDA	('Apex Asset', r'APEX ')
APEX ASSET MANAGEMENT LTDA	('Apex Asset', r'APEX ')
APEX CAPITAL LTDA	('Apex Capital', r'APEX ')
APO CAPITAL LTDA.	('Apo Capital', r'APO ')
APOLO ADMINISTRACAO DE RECURSOS LTDA	('Apolo Investimentos', r'APOLO ')
APOLO INVESTIMENTOS LTDA	('Apolo Investimentos', r'APOLO ')
APUAMA CAPITAL GESTORA DE RECURSOS LTDA	('Apuama Capital', r'APUAMA')
AQUA GESTÃO DE VALORES MOBILIÁRIOS LTDA	('Aqua', r'AQUA')
ARAM CAPITAL CONSULTORIA DE VALORES MOBILIÁRIOS LTDA.	('Aram Capital', r'ARAM')
ARAMUS GESTORA DE ATIVOS LTDA	('Aramus', r'ARAMUS')
ARANDU PORTFOLIO SOLUTIONS LTDA	('Arandu Investimentos', r'ARANDU ')
ARANDU SPECIAL SITUATIONS GESTÃO DE RECURSOS LTDA	('Arandu Investimentos', r'ARANDU ')
ARAZUL CAPITAL ASSET MANAGEMENT LTDA.	('Arazul', r'ARAZUL')
ARBI DIFERENCIAL ADMINISTRACAO E GESTAO DE RECURSOS LTDA	('Arbi', r'ARBI ')
ARBITRAL GESTÃO DE RECURSOS LTDA	('Arbitral', r'ARBITRAL')
ARBOR GESTAO DE RECURSOS FINANCEIROS LTDA	('Arbor Capital', r'ARBOR ')
ARBOR GESTÃO DE RECURSOS LTDA.	('Arbor Capital', r'ARBOR')
ARC CAPITAL LTDA.	('Arc Capital', r'ARC ')
ARCHIPELAGO INVESTIMENTOS LTDA	('Archipelago Investimentos', r'ARCHIPELAGO')
ARENA CAPITAL GESTORA DE RECURSOS LTDA	('Arena Capital', r'^ARENA ')
ARGOS FUNDOS DE INVESTIMENTOS LTDA.	('Argos ', r'ARGOS')
ARGUCIA CAPITAL GESTÃO DE RECURSOS LTDA	('Argucia Capital', r'ARGUCIA ')
ARGUMENTO ADMINISTRAÇÃO DE CARTEIRA DE TITULOS E VALORES MOBILIÁRIOS LTDA	('Argumento Gestão', r'ARGUMENTO ')
Ark Gestora de Recursos Ltda.	('Ark Investimentos', r'ARK ')
ARM CAPITAL Ltda.	('Arm Capital', r'ARM ')
ARMOR GESTORA DE RECURSOS LTDA.	('Armor Capital', r'ARMOR ')
AROEIRA ASSET GESTÃO DE RECURSOS LTDA	('Aroeira Asset', r'AROEIRA ')
ARQUIA ASSET GESTÃO E ADMINISTRAÇÃO DE INVESTIMENTOSA S.A.	('Arquia Asset', r'ARQUIA ')
ARSENAL INVESTIMENTOS LTDA	('Arsenal Investimentos', r'ARSENAL ')
ÁRTEMIS GESTÃO DE RECURSOS LTDA.	('Ártemis Gestão', r'ÁRTEMIS')
ARTESANAL FINANCEIRO LTDA	('Artesanal Investimentos', r'ARTESANAL ')
ARTESANAL INVESTIMENTOS LTDA	('Artesanal Investimentos', r'ARTESANAL ')
ARTESANAL MCMS LTDA	('Artesanal Investimentos', r'ARTESANAL ')
ARTICA GESTÃO DE RECURSOS LTDA	('Ártica Capital', r'ARTICA ')
ARTIS GESTORA DE RECURSOS S.A.	('Artis Gestora', r'ARTIS ')
ARTON GESTÃO DE RECURSOS LTDA	('Arton Advisors', r'ARTON ')
ARX INVESTIMENTOS LTDA	('Arx Investimentos', r'ARX ')
ARZ CAPITAL GESTORA DE RECURSOS LTDA.	('Arz Capital', r'ARZ ')
ASA ASSET MANAGEMENT LTDA.	('Asa Asset', r'ASA ')
ASAROCK ASSET MANAGEMENT LTDA.	('Asarock Asset', r'ASAROCK ')
ASHMORE BRASIL GESTORA DE RECURSOS LTDA	('Ashmore Group', r'ASHMORE ')
ASK GESTORA DE RECURSOS LTDA	('ASK - Advisory Services Kapital', r'ASK ')
ASQ ASSET MANAGEMENT LTDA.	('Asq Asset', r'ASQ ')
ASSET BANK ASSET MANAGEMENT LTDA.	('Asset Bank', r'ASSET BANK')
ASSET GF2 LTDA	('GF2 Investimentos', r'GF2 ')
ASSET1 INVESTIMENTOS S/A	('Asset 1', r'ASSET1')
ASTER ASSET MANAGEMENT S.A.	('Aster Capital', r'ASTER ')
ASTER INVESTIMENTOS S.A.	('Aster Capital', r'ASTER ')
ASTOR GESTÃO DE RECURSOS LTDA	('Astor Capital', r'ASTOR ')
ASTRA INVESTIMENTOS LTDA	('Astra Capital', r'ASTRA ')
ATENA CAPITAL GESTÃO DE RECURSOS LTDA.	('Atena Capital', r'ATENA ')
ATF GESTORA DE RECURSOS LTDA.	('ATF', r'ATF ')
ATHENA CAPITAL GESTÃO DE RECURSOS LTDA	('Athena Capital', r'ATHENA ')
ATIVA ASSET LTDA	('Ativa Asset', r'ATIVA ')
ATIVA INVESTIMENTOS S.A. C.T.C.V.	('Ativa Asset', r'ATIVA ')
ATIVORE GESTORA DE RECURSOS LTDA.	('Ativore Asset', r'ATIVORE ')
ATLÂNTICA ADMINISTRAÇÃO DE RECURSOS LTDA	('Atlântica Administração de Recursos', r'ATLÂNTICA ADMINISTRAÇÃO ')
ATLAS ONE INVESTIMENTOS GESTÃO DE RECURSOS LTDA.	('Atlas One', r'ATLAS ONE')
ATMOS CAPITAL GESTÃO DE RECURSOS LTDA	('Atmos Capital', r'ATMOS CAPITAL')
ATMOSPHERE CAPITAL GESTAO DE RECURSOS LTDA.	('Atmosphere Capital', r'ATMOSPHERE CAPITAL')
ATR GESTÃO DE RECURSOS FINANCEIROS LTDA	('Atr Gestão', r'ATR ')
ATUAL GESTORA DE RECURSOS LTDA.	('Atual Gestora', r'ATUAL GESTORA DE RECURSOS ')
AUGME CAPITAL GESTÃO DE RECURSOS LTDA.	('Augme Capital', r'AUGME ')
ÁUREO ADMINISTRAÇÃO DE RECURSOS LTDA	('Áureo Asset', r'ÁUREO ')
AURO CAPITAL LTDA	('Auro Capital', r'AURO ')
AURUM GESTÃO DE PATRIMÔNIO LTDA.	('Aurum Wealth Management', r'AURUM ')
AUSTER CAPITAL GESTÃO DE RECURSOS LTDA.	('Auster Capital', r'AUSTER CAPITAL')
AUSTRO GESTÃO DE RECURSOS LTDA.	('Austro Gestão', r'AUSTRO ')
AUTÊNTICO CAPITAL LTDA	('Autêntico Capital', r'AUTÊNTICO ')
AUTONOMY INVESTIMENTOS LTDA.	('Autonomy', r'AUTONOMY')
AVALON CAPITAL GESTORA DE RECURSOS LTDA.	('Avalon Capital', r'AVALON ')
AVANTGARDE ASSET MANAGEMENT GESTAO DE RECURSOS LTDA	('Avantgarde Asset', r'AVANTGARDE ')
AVAT INVESTIMENTOS GESTÃO DE VALORES MOBILIÁRIOS LTDA	('Avat Investimentos', r'AVAT ')
AVENTIS GESTÃO DE RECURSOS LTDA.	('Aventis ', r'AVENTIS ')
AVENUE SECURITIES GESTÃO DE RECURSOS LTDA.	('Avenue', r'AVENUE ')
AVVERO ASSET GESTAO DE RECURSOS LTDA	('Avvero Asset', r'AVVERO')
AWARE GESTÃO DE RECURSOS LTDA.	('Aware Investments', r'AWARE ')
AWR GESTÃO DE RECURSOS LTDA.	('AWR Capital', r'AWR ')
AWT CONSULTORIA E ADMINISTRAÇÃO DE VALORES MOBILIÁRIOS LTDA.	('AWT Group', r'AWT ')
AXIA GESTÃO DE INVESTIMENTOS EIRELI	('Axia Management', r'AXIA ')
AXIO INVESTIMENTOS GESTÃO DE RECURSOS LTDA	('Axio Investimentos', r'AXIO ')
AZ & PARTNERS GESTÃO DE RECURSOS LTDA.	('AZ Quest', r'^AZ ')
AZ GUIDANCE GESTÃO DE PATRIMÔNIO LTDA.	('AZ Quest', r'^AZ ')
AZ LEGAN ADMINISTRAÇÃO DE RECURSOS LTDA	('AZ Quest', r'^AZ ')
AZ QUEST INFRA LTDA.	('AZ Quest', r'^AZ ')
AZ QUEST INVESTIMENTOS LTDA.	('AZ Quest', r'^AZ ')
AZ QUEST MZK INVESTIMENTOS MACRO E CRÉDITO LTDA.	('AZ Quest', r'^AZ ')
AZ QUEST PANORAMA LTDA.	('AZ Quest', r'^AZ ')
AZ Quest Prev Ltda.	('AZ Quest', r'^AZ ')
AZIMUT BRASIL CONSULTORIA E CORRESPONDENTE BANCÁRIO LTDA	('AZ Quest', r'AZIMUT ')
AZIMUT BRASIL WEALTH MANAGEMENT LTDA	('AZ Quest', r'AZIMUT ')
AZUMI ASSET MANAGEMENT LTDA.	('Azumi Asset', r'AZUMI')
B&Z GLOBAL STRATEGIES GESTÃO DE INVESTIMENTOS LTDA	('B&Z GLOBAL STRATEGIES', r'B\&Z GLOBAL STRATEGIES')
B.SIDE WEALTH MANAGEMENT GESTÃO DE RECURSOS LTDA	('Bside', r'B.{0,3}SIDE')
B2V CRYPTO GESTORA DE RECURSOS LTDA.	('B2V Crypto', r'B2V CRYPTO')
B6 GESTORA DE RECURSOS LTDA	('B6 Capital', r'B6 ')
BAHIA AM RENDA FIXA LTDA	('Bahia Asset', r'BAHIA AM')
BAHIA AM RENDA VARIÁVEL LTDA	('Bahia Asset', r'BAHIA AM')
BALUARTE CAPITAL GESTÃO DE RECURSOS LTDA.	('Baluarte Capital', r'BALUARTE ')
BANCO ABC BRASIL S/A	('Abc', r'ABC')
BANCO ABN AMRO REAL S.A.	('Abn', r'ABN')
BANCO ALFA DE INVESTIMENTO S.A.	('Alfa', r'ALFA')
BANCO B3 S.A.	('B3', r'B3')
BANCO BNP PARIBAS BRASIL S/A	('Bnp', r'BNP')
BANCO BOCOM BBM S.A.	('Bocom', r'Bocom')
BANCO BRADESCO S.A.	('Bradesco', r'Bradesco')
BANCO BTG PACTUAL S/A	('BTG', r'BTG')
BANCO BVA SA	('Banco BVA', r'BVA')
BANCO C6 S.A.	('C6', r'C6')
BANCO CARGILL S.A.	('Cargill', r'CARGILL')
BANCO CITIBANK SA	('Citibank', r'Citibank')
BANCO CLÁSSICO S.A.	('Banco Clássico', r'BANCO CLÁSSICO')
BANCO COMERCIAL E DE INVESTIMENTO SUDAMERIS S.A.	('Sudameris', r'SUDAMERIS')
BANCO COOPERATIVO SICREDI S.A.	('Sicredi', r'SICREDI')
BANCO CREDIT AGRICOLE BRASIL S/A	('Credit', r'CREDIT')
BANCO DAYCOVAL S.A.	('Daycoval', r'Daycoval')




BANCO DE INVESTIMENTOS UBS (BRASIL) S.A.	('Ubs', r'UBS')
BANCO DO ESTADO DE SANTA CATARINA S/A - BESC	('Estado', r'ESTADO')
BANCO DO ESTADO DO RIO GRANDE DO SUL SA	('Estado', r'ESTADO')
BANCO DO NORDESTE DO BRASIL SA	('Nordeste', r'NORDESTE')
BANCO FATOR S/A	('Fator', r'FATOR')
BANCO GERDAU S.A	('Gerdau', r'GERDAU')
Banco HSBC S.A.	('Hsbc', r'HSBC')
BANCO INTERCAP S.A.	('Intercap', r'INTERCAP')
BANCO ITAUCARD S.A.	('Itaú', r'Ita(ú|u)')
BANCO J. SAFRA S.A.	('Safra', r'Safra')
BANCO J.P. MORGAN S.A.	('Jp', r'JP')
BANCO MIZUHO DO BRASIL S/A	('Mizuho', r'MIZUHO')
BANCO MODAL S.A.	('Modal', r'MODAL')
BANCO MUFG BRASIL S.A.	('Mufg', r'MUFG')
BANCO NOSSA CAIXA S/A	('Caixa', r'Caixa')
BANCO PAN SA	('Pan', r'PAN')
BANCO PAULISTA S.A.	('Paulista', r'PAULISTA')
BANCO PINE S/A	('Pine', r'PINE')
BANCO PLENO S.A.	('Pleno', r'PLENO')
BANCO RABOBANK INTERNATIONAL BRASIL S.A.	('Rabobank', r'RABOBANK')
BANCO RURAL S/A - EM LIQUIDAÇÃO EXTRAJUDICIAL	('Rural', r'RURAL')
BANCO SAFRA BSI S/A	('Safra', r'Safra')
BANCO SAFRA S/A	('Safra', r'Safra')
BANCO SANTANDER (BRASIL) S.A.	('Santander', r'Santander')
BANCO SANTANDER BRASIL SA	('Santander', r'Santander')
BANCO SOCIETE GENERALE BRASIL S.A.	('Societe', r'SOCIETE')
BANCO SUMITOMO MITSUI BRASILEIRO S.A.	('Sumitomo', r'SUMITOMO')
BANCO VOTORANTIM S.A.	('Votorantim', r'VOTORANTIM')
BANESTES DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.	('Banestes', r'BANESTES')
BANESTES SEGUROS S/A	('Banestes', r'BANESTES')
BANIF GESTÃO DE ATIVOS (BRASIL) S.A.	('Banif', r'BANIF')
BANK OF AMERICA MERRILL LYNCH BANCO MULTIPLO S.A.	('BofA', r'Bank of America')
BANK OF CHINA (BRASIL) BANCO MÚLTIPLO S.A.	('China', r'CHINA')
BANRISUL S.A. CORRETORA DE VALORES MOBILIÁRIOS E CÂMBIO	('Banrisul', r'BANRISUL')
BARRA PEIXE INVESTIMENTOS LTDA.	('Barra', r'BARRA')
BAWM GESTÃO DE RECURSOS LTDA.	('Bawm', r'BAWM')
BAYES CAPITAL MANAGEMENT INVESTIMENTOS LTDA.	('Bayes', r'BAYES')
BB GESTAO DE RECURSOS DTVM S.A	('BB', r'BB')
BBM GESTÃO DE RECURSOS LTDA.	('Bbm', r'BBM')
BC GESTÃO DE RECURSOS LTDA	('Bc', r'BC')
BCSUL VERAX SERVIÇOS FINANCEIROS LTDA	('Bcsul', r'BCSUL')
BCV - BANCO DE CRÉDITO E VAREJO S.A.	('Bcv', r'BCV')
BDR INVESTIMENTOS LTDA.	('Bdr', r'BDR')
BEHAVIOR GESTÃO DE CAPITAL LTDA	('Behavior', r'BEHAVIOR')
BELVEDERE ADMINISTRAÇÃO DE VALORES MOBILIÁRIOS LTDA.	('Belvedere', r'BELVEDERE')
BEM - DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA.	('Bem', r'BEM')
BENAVENT ASSET LTDA	('Benavent', r'BENAVENT')
BER CAPITAL S.A.	('Ber', r'BER')
BERTHA CAPITAL GESTORA DE RECURSOS S.A.	('Bertha', r'BERTHA')
BFL ADMINISTRAÇÃO DE RECURSOS LTDA	('Bfl', r'BFL')
BI CAPITAL GESTAO DE RECURSOS LTDA	('Bi', r'BI')
BIGUÁ CAPITAL GESTÃO DE RECURSOS LTDA	('Biguá', r'BIGUÁ')
BLACK SWAN GESTÃO DE INVESTIMENTOS LTDA.	('Black', r'BLACK')
BLACKROCK BRASIL GESTORA DE INVESTIMENTOS LTDA	('Blackrock', r'BLACKROCK')
BLESS CAPITAL GESTORA DE RECURSOS LTDA.	('Bless', r'BLESS')
BLOXS ASSET GESTÃO DE RECURSOS LTDA	('Bloxs', r'BLOXS')
BLP CAPITAL GESTORA DE RECURSOS LTDA.	('Blp', r'BLP')
BLUE ASSET GESTÃO DE RECURSOS LTDA.	('Blue', r'BLUE')
BLUE OCEAN GESTÃO DE RECURSOS FINANCEIROS LTDA.	('Blue', r'BLUE')
BLUE SOLUTIONS ASSET MANAGEMENT LTDA.	('Blue', r'BLUE')
BLUE STAR E ASSET MANAGEMENT LTDA	('Blue', r'BLUE')
BLUELINE ASSET MANAGEMENT S.A.	('Blueline', r'BLUELINE')
BLUEMAC ASSET MANAGEMENT LTDA	('Bluemac', r'BLUEMAC')
BLUEMETRIX GESTAO DE ATIVOS S.A.	('Bluemetrix', r'BLUEMETRIX')
BLUEOAK INVESTMENTS ASSET LTDA.	('Blueoak', r'BLUEOAK')
BLUEWAVE ASSET LTDA	('Bluewave', r'BLUEWAVE')
BMC ASSET MANAGEMENT - DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Bmc', r'BMC')
BNP PARIBAS ASSET MANAGEMENT BRASIL LTDA	('Bnp', r'BNP')
BNY MELLON ADMINISTRAÇÃO DE ATIVOS LTDA	('Bny', r'BNY')
BNY MELLON ALOCAÇÃO DE PATRIMÔNIO LTDA	('Bny', r'BNY')
BNY MELLON SERVICOS FINANCEIROS DTVM S.A.	('Bny', r'BNY')
BOCAINA CAPITAL GESTORA DE RECURSOS LTDA.	('Bocaina', r'BOCAINA')
BOCOM BBM CCVM S.A.	('Bocom', r'Bocom')
BOGARI GESTÃO DE INVESTIMENTOS LTDA.	('Bogari', r'BOGARI')
BORDUNA CAPITAL GESTAO DE RECURSOS LTDA	('Borduna', r'BORDUNA')
BOREAL HOLDING S.A.	('Boreal', r'BOREAL')
BOX ASSET MANAGEMENT GESTORA DE RECURSOS LTDA.	('Box', r'BOX')
BR-CAPITAL DTVM S.A.	('Brcapital', r'BRCAPITAL')
BR PARTNERS BANCO DE INVESTIMENTO S.A.	('Br', r'BR')
BR PARTNERS GESTÃO DE RECURSOS LTDA	('Br', r'BR')
BRADESCO VIDA E PREVIDÊNCIA S.A.	('Bradesco', r'Bradesco')
BRAINVEST ASSESSORIA FINANCEIRA E GESTÃO DE RECURSOS LTDA	('Brainvest', r'BRAINVEST')
BRAM - BRADESCO ASSET MANAGEMENT LTDA	('Bradesco', r'Bradesco')
BRAM - BRADESCO ASSET MANAGEMENT S.A. DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS	('Bradesco', r'Bradesco')
BRASIF GESTÃO INTERNACIONAL LTDA	('Brasif', r'BRASIF')
BRASIL GLOBAL PARTNERS LTDA.	('Brasil', r'BRASIL')
BRASILPREV SEGUROS E PREVIDÊNCIA S.A	('Brasilprev', r'BRASILPREV')
BRAVE GESTORA DE RECURSOS LTDA.	('Brave', r'BRAVE')
BRAVIA IMPACT ASSETS LTDA	('Bravia', r'BRAVIA')
BRAX INVESTIMENTOS LTDA	('Brax', r'BRAX')
BRB DTVM SA	('Brb', r'BRB')
BRBK PARTICIPAÃiES S.A.	('Brbk', r'BRBK')
BRDR GESTORA DE VALORES MOBILIÁRIOS LTDA.	('Brdr', r'BRDR')
BREI - BRAZILIAN REAL ESTATE INVESTMENTS LTDA	('Brei', r'BREI')
BRESCO INVESTIMENTOS E GESTÃO LTDA	('Bresco', r'BRESCO')
BRESSER ADMINISTRAÇÃO DE RECURSOS LTDA	('Bresser', r'BRESSER')
BRICK CAPITAL LTDA.	('Brick', r'BRICK')
BRIDGE GESTORA DE RECURSOS LTDA.	('Bridge', r'BRIDGE')
BRL TRUST DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A.	('Brl', r'BRL')
BRL TRUST INVESTIMENTOS LTDA.	('Brl', r'BRL')
BRL TRUST SERVIÇOS FIDUCIÁRIOS E PARTICIPAÇÕES LTDA	('Brl', r'BRL')
BRM APEX SOLUCOES DE PORTFOLIO GESTAO DE RECURSOS LTDA.	('Brm', r'BRM')
BRM CARBYNE GESTÃO DE RECURSOS LTDA.	('Brm', r'BRM')
BROADEN CAPITAL GESTÃO DE INVESTIMENTOS LTDA	('Broaden', r'BROADEN')
BROOKFIELD GESTÃO DE ATIVOS LTDA	('Brookfield', r'BROOKFIELD')
BRZ INVESTIMENTOS LTDA	('Brz', r'BRZ')
BS2 ASSET MANAGEMENT - ADMINISTRADORA DE RECURSOS LTDA	('Bs2', r'BS2')
BTG PACTUAL  GESTÃO E CONSULTORIA DE INVESTIMENTOS LTDA	('BTG', r'BTG')
BTG PACTUAL ASSET MANAGEMENT S/A DTVM	('BTG', r'BTG')
BTG PACTUAL GESTORA DE INVESTIMENTOS ALTERNATIVOS LTDA.	('BTG', r'BTG')
BTG PACTUAL GESTORA DE RECURSOS LTDA	('BTG', r'BTG')
BTG PACTUAL SERVIÇOS FINANCEIROS S/A DTVM	('BTG', r'BTG')
BTG PACTUAL WM GESTÃO DE RECURSOS LTDA	('BTG', r'BTG')
BTGP WM 3 LTDA	('BTG', r'BTG')
BUENA VISTA GESTORA DE RECURSOS LTDA.	('Buena', r'BUENA')
BURITI INVESTIMENTOS GESTORA DE RECURSOS LTDA	('Buriti', r'BURITI')
BUTIÁ GESTÃO DE INVESTIMENTOS LTDA.	('Butiá', r'BUTIÁ')
BW GESTÃO DE INVESTIMENTOS LTDA.	('Bw', r'BW')
BWAG GESTÃO DE RECURSOS LTDA	('Bwag', r'BWAG')
BY TRUST GESTAO DE ATIVOS LTDA	('By', r'BY')
C Q PARTICIPAÇÕES LTDA	('C', r'C')
CADENCE GESTORA DE RECURSOS LTDA	('Cadence', r'CADENCE')
CAIXA BENEFICENTE DOS EMPREGADOS DA CSN - CBS	('Caixa', r'Caixa')
CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO BRASIL - PREVI	('Caixa', r'Caixa')
CAIXA DE PREVIDÊNCIA DOS FUNCIONÁRIOS DO BANCO DO NORDESTE DO BRASIL	('Caixa', r'Caixa')
CAIXA DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.	('Caixa', r'Caixa')
CAIXA ECONOMICA FEDERAL	('Caixa', r'Caixa')
CAIXA SEGURADORA SA	('Caixa', r'Caixa')
CAIXA VIDA E PREVIDÊNCIA S/A	('Caixa', r'Caixa')
CAMBUHY INVESTIMENTOS LTDA	('Cambuhy', r'CAMBUHY')
CANUMA CAPITAL LTDA.	('Canuma', r'CANUMA')
CANVAS CAPITAL S.A.	('Canvas', r'CANVAS')
CAPEMISA SEGURADORA DE VIDA E PREVIDÊNCIA S/A	('Capemisa', r'CAPEMISA')
CAPITAL GESTÃO E INVESTIMENTOS LTDA	('Capital', r'CAPITAL')
CAPITALPLUS GESTÃO DE ATIVOS S/A	('Capitalplus', r'CAPITALPLUS')
CAPITÂNIA ALTERNATIVES S/A	('Capitânia', r'CAPITÂNIA')
CAPITANIA CAPITAL S.A.	('Capitania', r'CAPITANIA')
CAPITÂNIA INVEST S/A	('Capitânia', r'CAPITÂNIA')
CAPITANIA PREV S.A.	('Capitania', r'CAPITANIA')
CAPRI INVESTIMENTOS LTDA.	('Capri', r'CAPRI')
CAPSIGMA INVESTMENT PARTNERS LTDA.	('Capsigma', r'CAPSIGMA')
CAPSTONE PARTNERS GESTÃO DE RECURSOS LTDA.	('Capstone', r'CAPSTONE')
CAPSUR CAPITAL GESTÃO DE RECURSOS LTDA.	('Capsur', r'CAPSUR')
CARAVELA CONSULTORIA E GESTÃO DE INVESTIMENTOS LTDA.	('Caravela', r'CARAVELA')
CARBON ASSET MANAGEMENT LTDA.	('Carbon', r'CARBON')
CARDINAL PARTNERS INVESTIMENTOS LTDA.	('Cardinal', r'CARDINAL')
CARMEL GESTORA DE ATIVOS LTDA	('Carmel', r'CARMEL')
CARPA GESTORA DE RECURSOS LTDA.	('Carpa', r'CARPA')
CARTESIA INVESTIMENTOS E GESTÃO DE RECURSOS LTDA	('Cartesia', r'CARTESIA')
CARTOR CAPITAL GESTÃO DE RECURSOS LTDA.	('Cartor', r'CARTOR')
CATÁLISE INVESTIMENTOS LTDA.	('Catálise', r'CATÁLISE')
CATALUNYA GESTÃO DE RECURSOS LTDA.	('Catalunya', r'CATALUNYA')
CATUAÍ GESTORA DE RECURSOS LTDA.	('Catuaí', r'CATUAÍ')
CCF INVESTIMENTOS LTDA	('Ccf', r'CCF')
CEDARWOOD PRIVATE EQUITY S.A.	('Cedarwood', r'CEDARWOOD')
CEDRO ASSET MANAGEMENT LTDA	('Cedro', r'CEDRO')
CENARIO 2003 INVESTIMENTOS E ADMINISTRAÇÃO DE RECURSOS LTDA	('Cenario', r'CENARIO')
CENTRAL CAPITAL GESTAO DE RECURSOS S.A.	('Central', r'CENTRAL')
CERES - FUNDAÇÃO DE PREVIDÊNCIA	('Ceres', r'CERES')
CERES ASSET GESTAO DE INVESTIMENTOS LTDA	('Ceres', r'CERES')
CFO ADMINISTRACAO DE RECURSOS LTDA	('Cfo', r'CFO')
CHARLES RIVER ADMINISTRADORA DE RECURSOS FINANCEIROS LTDA.	('Charles', r'CHARLES')
CHESS CAPITAL LTDA.	('Chess', r'CHESS')
CHIMERA CAPITAL ASSET MANAGEMENT LTDA.	('Chimera', r'CHIMERA')
CHROMO INVESTIMENTOS LTDA.	('Chromo', r'CHROMO')
CIANO GESTÃO DE RECURSOS E EDUCAÇÃO LTDA.	('Ciano', r'CIANO')
CIBRIUS - INSTITUTO CONAB DE SEGURIDADE SOCIAL	('Cibrius', r'CIBRIUS')
CITIBANK DTVM SA	('Citibank', r'Citibank')
CITIGROUP GLOBAL MARKETS BRASIL C.C.T.V.M. S.A.	('Citibank', r'Citi')
CITRINO GESTÃO DE RECURSOS LTDA	('Citrino', r'CITRINO')
CIX CAPITAL GESTÃO DE ATIVOS LTDA.	('Cix', r'CIX')
CL4 CAPITAL GESTORA DE RECURSOS LTDA.	('Cl4', r'CL4')
CLARITAS SERVICOS FINANCEIROS LTDA	('Claritas', r'CLARITAS')
CLAVE ALTERNATIVOS GESTORA DE RECURSOS LTDA.	('Clave', r'CLAVE')
CLAVE GESTORA DE RECURSOS LTDA.	('Clave', r'CLAVE')
CLICKTRADE CCTVM S.A.	('Clicktrade', r'CLICKTRADE')
CLUBE DO VALOR ADMINISTRAÇÃO DE CARTEIRAS DE VALORES MOBILIÁRIOS LTDA	('Clube', r'CLUBE')
CMS GESTAO E ASSESSORIA LTDA	('Cms', r'CMS')
COGNUS CAPITAL GESTÃO DE RECURSOS LTDA.	('Cognus', r'COGNUS')
COINEXT GESTÃO DE RECURSOS LTDA.	('Coinext', r'COINEXT')
COINVALORES CCVM LTDA	('Coinvalores', r'COINVALORES')
COLUMBOS PARTICIPAÇÕES LTDA.	('Columbos', r'COLUMBOS')
COMERCIAL ASSET MANAGEMENT ADMINISTRAÇÃO DE RECURSOS S.A.	('Comercial', r'COMERCIAL')
COMPOSTELA CAPITAL GESTORA DE RECURSOS LTDA	('Compostela', r'COMPOSTELA')
COMPOUND PARTNERS GESTAO DE RECURSOS LTDA	('Compound', r'COMPOUND')
CONCÓRDIA GESTÃO DE RECURSOS LTDA	('Concórdia', r'CONCÓRDIA')
CONCRETA GESTORA DE RECURSOS LTDA.	('Concreta', r'CONCRETA')
CONFEDERAÇÃO INTERESTADUAL DAS COOPERATIVAS LIGADAS AO SICREDI	('Confederação', r'CONFEDERAÇÃO')
CONFIDARE ASSET MANAGEMENT ADMINISTRACAO DE RECURSOS LTDA	('Confidare', r'CONFIDARE')
CONNEX GESTORA DE RECURSOS LTDA.	('Connex', r'CONNEX')
CONSTÂNCIA INVESTIMENTOS LTDA.	('Constância', r'CONSTÂNCIA')
CONSTELLATION INVESTIMENTOS E PARTICIPAÇÕES LTDA	('Constellation', r'CONSTELLATION')
CONTEA CAPITAL GESTAO DE RECURSOS LTDA	('Contea', r'CONTEA')
CONVERGE CAPITAL GESTÃO DE RECURSOS LTDA	('Converge', r'CONVERGE')
CONVEST CONSULTORIA DE INVESTIMENTOS LTDA	('Convest', r'CONVEST')
COPACABANA GESTÃO DE RECURSOS FINANCEIROS LTDA.	('Copacabana', r'COPACABANA')
CORCOVADO INVESTIMENTOS GESTÃO DE RECURSOS LTDA.	('Corcovado', r'CORCOVADO')
CORE CAPITAL GESTORA DE RECURSOS LTDA.	('Core', r'CORE')
COX GESTÃO DE RECURSOS LTDA	('Cox', r'COX')
CPV CAPITAL GESTÃO DE RECURSOS LTDA.	('Cpv', r'CPV')
CR2 SERVIÇOS FINANCEIROS LTDA	('Cr2', r'CR2')
CRD CAPITAL ADMINISTRAÇÃO DE RECURSOS LTDA	('Crd', r'CRD')
CREDIT SUISSE (BRASIL) DTVM S/A	('Credit Suisse', r'Credit Suisse')
CREDIT SUISSE (BRASIL) S/A CTVM	('Credit Suisse', r'Credit Suisse')
CREDIT SUISSE CONSULTORIA DE INVESTIMENTOS LTDA	('Credit Suisse', r'Credit Suisse')
CREDIT SUISSE HEDGING-GRIFFO WEALTH MANAGEMENT S.A.	('Credit Suisse', r'Credit Suisse')
CRESCERA GROWTH CAPITAL LTDA.	('Crescera', r'CRESCERA')
CRP COMPANHIA DE PARTICIPACOES	('Crp', r'CRP')
CRPC ADMINISTRAÇÃO DE RECURSOS FINANCEIROS LTDA.	('Crpc', r'CRPC')
CULTINVEST ASSET MANAGEMENT LTDA	('Cultinvest', r'CULTINVEST')
CUPERTINO CAPITAL GESTÃO DE RECURSOS LTDA	('Cupertino', r'CUPERTINO')
CVPAR INVESTIMENTOS LTDA	('Cvpar', r'CVPAR')
CX3 CONSULTORIA E GESTAO DE INVESTIMENTOS LTDA.	('Cx3', r'CX3')
CYGNUS ASSET GESTÃO DE RECURSOS LTDA	('Cygnus', r'CYGNUS')
D3 CAPITAL GESTORA DE RECURSOS LTDA	('D3', r'D3')
DAEMON INVESTIMENTOS LTDA.	('Daemon', r'DAEMON')
DAHLIA CAPITAL GESTÃO DE RECURSOS LTDA.	('Dahlia', r'DAHLIA')
DAO CAPITAL LTDA.	('Dao', r'DAO')
DAYCOVAL ASSET MANAGEMENT ADMINISTRACAO DE RECURSOS LTDA	('Daycoval', r'Daycoval')
DC 1000 CONSULTORIA FINANCEIRA LTDA.	('Dc', r'DC')
DELOS ASSET MANAGEMENT ADMINISTRADORA DE CARTEIRAS DE VALORES MOBILIÁRIOS LTDA	('Delos', r'DELOS')
DELTA ENERGIA ADMINISTRAÇÃO DE RECURSOS LTDA.	('Delta', r'DELTA')
DELTA ENERGIA ASSET MANAGEMENT - GESTÃO DE RECURSOS LTDA.	('Delta', r'DELTA')
DELTA HEDGE EMPREENDIMENTOS E CONSULTORIA ECONÔMICO FINANCEIRA LTDA	('Delta', r'DELTA')
DESBRAVA CAPITAL LTDA	('Desbrava', r'DESBRAVA')
DEUTSCHE BANK SA	('Deutsche', r'DEUTSCHE')
DEVANT ASSET INVESTIMENTOS LTDA	('Devant', r'DEVANT')
DEX CAPITAL GESTÃO DE RECURSOS LTDA.	('Dex', r'DEX')
DEZESSEIS DEZOITO GESTÃO DE RECURSOS LTDA	('Dezesseis', r'DEZESSEIS')
DG ADMINISTRADORA DE CARTEIRAS DE VALORES MOBILIARIOS LTDA	('Dg', r'DG')
DGF INVESTIMENTOS GESTÃO DE FUNDOS LTDA	('Dgf', r'DGF')
DIFERENCIAL CONSULTORIA E ASSESSORIA EMPRESARIAL LTDA	('Diferencial', r'DIFERENCIAL')
DIRETA CAPITAL LTDA	('Direta', r'DIRETA')
DIVERSA CAPITAL GESTÃO DE RECURSOS LTDA	('Diversa', r'DIVERSA')
DIVITIA INVESTIMENTOS LTDA	('Divitia', r'DIVITIA')
DMI INVESTIMENTOS E GESTAO DE RECURSOS LTDA	('Dmi', r'DMI')
DNA CAPITAL CONSULTORIA LTDA	('Dna', r'DNA')
DOMINIO S/A PARTICIPAÇÕES	('Dominio', r'DOMINIO')
DOMO INVEST GESTORA DE ATIVOS FINANCEIROS E VALORES MOBILIÁRIOS LTDA.	('Domo', r'DOMO')
DOT. INVESTIMENTOS LTDA.	('Dot', r'DOT')
DOURO CAPITAL GESTORA DE RECURSOS E INVESTIMENTOS LTDA.	('Douro', r'DOURO')
DOXOS GESTÃO DE RECURSOS LTDA.	('Doxos', r'DOXOS')
DRACHMA INVESTIMENTOS S.A	('Drachma', r'DRACHMA')
DRYS CAPITAL LTDA	('Drys', r'DRYS')
DSK CAPITAL GESTORA DE RECURSOS E CONSULTORIA LTDA.	('Dsk', r'DSK')
DWN GESTÃO DE RECURSOS LTDA	('Dwn', r'DWN')
DXA GESTÃO DE INVESTIMENTOS S.A.	('Dxa', r'DXA')
DYNAMO ADMINISTRAÇÃO DE RECURSOS LTDA	('Dynamo', r'DYNAMO')
DYNAMO INTERNACIONAL GESTÃO DE RECURSOS LTDA.	('Dynamo', r'DYNAMO')
E2M INVESTIMENTOS LTDA.	('E2m', r'E2M')
E3 CAPITAL PARTNERS GESTÃO DE INVESTIMENTOS LTDA.	('E3', r'E3')
EAGLE CAPITAL GESTÃO DE INVESTIMENTOS LTDA	('Eagle', r'EAGLE')
EB CAPITAL CRÉDITO AGRO LTDA.	('Eb', r'EB')
EB CAPITAL GESTÃO DE RECURSOS LTDA.	('Eb', r'EB')
ECO GESTÃO DE ATIVOS LTDA	('Eco', r'ECO')
EFFIKA INVESTIMENTOS - GESTORA DE PATRIMÔNIO LTDA	('Effika', r'EFFIKA')
EKHO INVESTIMENTOS LTDA.	('Ekho', r'EKHO')
ELITE CCVM LTDA	('Elite', r'ELITE')
ELLEVEN GESTORA DE RECURSOS LTDA	('Elleven', r'ELLEVEN')
ELLO GESTORA DE RECURSOS LTDA	('Ello', r'ELLO')
EMERALD GESTÃO DE INVESTIMENTOS SOCIEDADE UNIPESSOAL LIMITADA	('Emerald', r'EMERALD')
EMPIRICUS ASSET MANAGEMENT  LTDA.	('Empiricus', r'EMPIRICUS')
ENCORE GESTÃO DE RECURSOS LTDA.	('Encore', r'ENCORE')
ENDURO CAPITAL INVESTIMENTOS LTDA	('Enduro', r'ENDURO')
ENSO GESTÃO DE RECURSOS LTDA.	('Enso', r'ENSO')
ENTERCAPITAL GESTÃO DE RECURSOS LTDA	('Entercapital', r'ENTERCAPITAL')
EOS INVESTIMENTOS LTDA.	('Eos', r'EOS')
EQUADOR INVESTIMENTOS LTDA.	('Equador', r'EQUADOR')
EQUITY GESTAO DE RECURSOS LTDA	('Equity', r'EQUITY')
EQUUS CAPITAL GESTORA DE RECURSOS LTDA	('Equus', r'EQUUS')
ERFOLG GESTORA DE RECURSOS S/A	('Erfolg', r'ERFOLG')
ESH CAPITAL INVESTIMENTOS LTDA	('Esh', r'ESH')
ESPIRITO SANTO PARTICIPACOES LTDA	('Espirito', r'ESPIRITO')
EST GESTÃO DE PATRIMÔNIO LTDA.	('Est', r'EST')
ETERNIA GESTORA DE RECURSOS LTDA.	('Eternia', r'ETERNIA')
ÉTRE-DAVOS GESTORA DE RECURSOS LTDA.	('Étredavos', r'ÉTREDAVOS')
ETRNTY CAPITAL GESTORA DE RECURSOS LTDA	('Etrnty', r'ETRNTY')
EUCALYPTUS INVESTIMENTOS LTDA	('Eucalyptus', r'EUCALYPTUS')
EUQUEROINVESTIR GESTÃO DE RECURSOS LTDA	('Euqueroinvestir', r'EUQUEROINVESTIR')
EURO GESTAO DE RECURSOS LTDA	('Euro', r'EURO')
EUROPA GESTÃO DE RECURSOS LTDA.	('Europa', r'EUROPA')
EUROVEST ASSET MANAGEMENT LTDA	('Eurovest', r'EUROVEST')
EVEREST CAPITAL GESTORA DE RECURSOS LTDA.	('Everest', r'EVEREST')
EVEREST CAPITAL OFFICE LTDA.	('Everest', r'EVEREST')
EVOLVE CAPITAL GESTÃO DE RECURSOS LTDA.	('Evolve', r'EVOLVE')
EXA CAPITAL ASSET LTDA.	('Exa', r'EXA')
EXANTE ASSET MANAGEMENT LTDA.	('Exante', r'EXANTE')
EXES GESTORA DE RECURSOS LTDA.	('Exes', r'EXES')
EXPLORITAS ADMINISTRACAO FINANCEIRA LTDA	('Exploritas', r'EXPLORITAS')
EXPRESSO PLANEJAMENTO GESTÃO DE RECURSOS LTDA	('Expresso', r'EXPRESSO')
EXT CAPITAL LTDA.	('Ext', r'EXT')
F3 GESTAO DE INVESTIMENTOS LTDA	('F3', r'F3')
F3 ROCK GESTÃO DE RECURSOS LTDA	('F3', r'F3')
F8 CAPITAL LTDA.	('F8', r'F8')
FABULA CAPITAL GESTÃO DE RECURSOS LTDA.	('Fabula', r'FABULA')
FACT INVESTMENTS GESTAO DE RECURSOS LTDA	('Fact', r'FACT')
FAMA PRIVATE EQUITY ADMINISTRAÃ+O DE CARTEIRA DE VALORES MOBILI-RIOS LTDA.	('Fama', r'FAMA')
FAMA RE.CAPITAL LTDA	('Fama', r'FAMA')
FAR FATOR ADMINISTRAÇÃO DE RECURSOS LTDA	('Far', r'FAR')
FARIA LIMA CAPITAL LTDA.	('Faria', r'FARIA')
FARMTECH GESTÃO DE RECURSOS LTDA.	('Farmtech', r'FARMTECH')
FARVIEW INVESTIMENTOS LTDA.	('Farview', r'FARVIEW')
FCL CAPITAL GESTÃO DE RECURSOS DE TERCEIROS LTDA.	('Fcl', r'FCL')
FIBRA ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIÁRIOS LTDA	('Fibra', r'FIBRA')
FIBRA ASSET MANAGEMENT DTVM LTDA.	('Fibra', r'FIBRA')
FICTOR ASSET LTDA.	('Fictor', r'FICTOR')
FIDAM GESTÃO DE INVESTIMENTOS LTDA	('Fidam', r'FIDAM')
FIDD ADMINISTRAÇÃO DE RECURSOS LTDA	('Fidd', r'FIDD')
FIDUC GESTÃO FIDUCIÁRIA S.A.	('Fiduc', r'FIDUC')
FIDUCIA ASSET MANAGEMENT LTDA	('Fiducia', r'FIDUCIA')
FIERE INVESTIMENTOS LTDA	('Fiere', r'FIERE')
FINACAP INVESTIMENTOS LTDA	('Finacap', r'FINACAP')
FINAXIS CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.	('Finaxis', r'FINAXIS')
FINHEALTH GESTAO DE RECURSOS LTDA	('Finhealth', r'FINHEALTH')
FINISTERE ASSET MANAGEMENT LTDA	('Finistere', r'FINISTERE')
FIRST WHO CAPITAL ASSET MANAGEMENT LTDA.	('First', r'FIRST')
FL GESTORA DE RECURSOS LTDA.	('Fl', r'FL')
FLORIDA INVESTIMENTOS E GESTÃO DE RECURSOS LTDA	('Florida', r'FLORIDA')
FOCUS ASSESSORIA EM INVESTIMENTO LTDA	('Focus', r'FOCUS')
FORNAX CONSULTORIA EMPRESARIAL S.A. (ATUAL DENOMINAÇÃO DA FMD GESTÃO DE RECURSOS	('Fornax', r'FORNAX')
FORPUS CAPITAL GESTÃO DE RECURSOS LTDA.	('Forpus', r'FORPUS')
FORTIOR GESTÃO DE PATRIMÔNIO LTDA	('Fortior', r'FORTIOR')
FORTIS GESTORA DE RECURSOS LTDA.	('Fortis', r'FORTIS')
FORTUNE GESTORA DE RECURSOS LTDA.	('Fortune', r'FORTUNE')
FOURCE GESTÃO DE RECURSOS LTDA.	('Fource', r'FOURCE')
FOX INVESTIMENTOS LTDA	('Fox', r'FOX')
FRAM CAPITAL EQUITIES - GESTÃO DE ATIVOS LTDA.	('Fram', r'FRAM')
FRAM CAPITAL GESTÃO DE ATIVOS LTDA.	('Fram', r'FRAM')
FRANKLIN TEMPLETON INVESTIMENTOS (BRASIL) LTDA	('Franklin', r'FRANKLIN')
FRONTEIRA - GESTÃO DE INVESTIMENTOS LTDA	('Fronteira', r'FRONTEIRA')
FRONTIER CAPITAL GESTÃO DE RECURSOS LTDA	('Frontier', r'FRONTIER')
FUNDAÇÃO ATLÂNTICO DE SEGURIDADE SOCIAL	('Fundação', r'FUNDAÇÃO')
FUNDACAO CESP	('Fundacao', r'FUNDACAO')
FUNDACAO CHESF DE ASSISTENCIA E SEGURIDADE SOCIAL - FACHESF	('Fundacao', r'FUNDACAO')
FUNDAÇÃO COPEL DE PREVIDÊNCIA E ASSISTÊNCIA SOCIAL	('Fundação', r'FUNDAÇÃO')
FUNDAÇÃO DE ASSISTÊNCIA E PREVIDÊNCIA SOCIAL DO BNDES - FAPES	('Fundação', r'FUNDAÇÃO')
FUNDAÇÃO DE PREVIDENCIA DOS SERVIDORES DO IRB - PREVIRB	('Fundação', r'FUNDAÇÃO')
FUNDACAO DE SEGURIDADE SOCIAL BRASLIGHT	('Fundacao', r'FUNDACAO')
FUNDAÇÃO DOS ECONOMIÁRIOS FEDERAIS - FUNCEF	('Fundação', r'FUNDAÇÃO')
FUNDAÇÃO ELETROBRÁS DE SEGURIDADE SOCIAL - ELETROS	('Fundação', r'FUNDAÇÃO')
FUNDACAO ENERSUL	('Fundacao', r'FUNDACAO')
FUNDAÇÃO PETROBRAS DE SEGURIDADE SOCIAL - PETROS	('Fundação', r'FUNDAÇÃO')
FUNDACAO SABESP DE SEGURIDADE SOCIAL - SABESPREV	('Fundacao', r'FUNDACAO')
FUNDAÇÃO SISTEL DE SEGURIDADE SOCIAL	('Fundação', r'FUNDAÇÃO')
FUNDAÇÃO VALE DO RIO DOCE DE SEGURIDADE SOCIAL - VALIA	('Fundação', r'FUNDAÇÃO')
FUNDAMENTA ADMINISTRAÇÃO DE CARTEIRAS DE VALORES MOBILIARIOS LTDA	('Fundamenta', r'FUNDAMENTA')
FUNDAMENTAL INVESTIMENTOS LTDA.	('Fundamental', r'FUNDAMENTAL')
FUNDIÁGUA - FUNDAÇÃO DE PREVIDÊNCIA DOS EMPREGADOS DA CAESB	('Fundiágua', r'FUNDIÁGUA')
FUNDO GARANTIDOR DE CRÉDITO - FGG	('Garantidor', r'GARANTIDOR')
FUNPRESP-JUD FUNDAÇÃO DE PREVIDÊNCIA COMPLEMENTAR DO SERVIDOR PÚBLICO FEDERAL DO PODER JUDICIÁRIO	('Funprespjud', r'FUNPRESPJUD')
FYTO CAPITAL ADMINISTRADORA DE RECURSOS LTDA	('Fyto', r'FYTO')
G.M INVESTMENT GESTORA DE RECURSOS LTDA.	('Gm', r'GM')
G10 ADMINISTRADORA DE RECURSOS LTDA	('G10', r'G10')
G5 ADMINISTRADORA DE RECURSOS LTDA.	('G5', r'G5')
G5 ASSESSORIA FINANCEIRA LTDA	('G5', r'G5')
G5 GESTORA DE RECURSOS LTDA	('G5', r'G5')
GAA GLOBAL ASSET ALLOCATION ADMINISTRACAO DE RECURSOS LTDA	('Gaa', r'GAA')
GALAPAGOS CAPITAL INVESTIMENTOS E PARTICIPAÇÕES LTDA.	('Galapagos', r'GALAPAGOS')
GALAPAGOS INVESTMENTS SOLUTIONS LTDA.	('Galapagos', r'GALAPAGOS')
GALAPAGOS MULTI FAMILY OFFICE LTDA	('Galapagos', r'GALAPAGOS')
GALAPAGOS WEALTH MANAGEMENT GESTÃO DE INVESTIMENTOS LTDA	('Galapagos', r'GALAPAGOS')
GALLOWAY CAPITAL GESTORA DE RECURSOS LTDA.	('Galloway', r'GALLOWAY')
GALOP CAPITAL LTDA.	('Galop', r'GALOP')
GALOPPO GESTÃO DE RECURSOS LTDA	('Galoppo', r'GALOPPO')
GALT CAPITAL CONSULTORIA DE INVESTIMENTOS LTDA.	('Galt', r'GALT')
GALVÃO ADMINISTRADORA DE RECURSOS LTDA.	('Galvão', r'GALVÃO')
GAMA INVESTIMENTOS LTDA.	('Gama', r'GAMA')
GAP ASSET MANAGEMENT S/C LTDA	('Gap', r'GAP')
GAP GESTORA DE RECURSOS LTDA	('Gap', r'GAP')
GARDE ASSET MANAGEMENT GESTÃO DE RECURSOS LTDA	('Garde', r'GARDE')
GARIN INVESTIMENTOS LTDA	('Garin', r'GARIN')
GATEINVEST GESTÃO DE RECURSOS LTDA.	('Gateinvest', r'GATEINVEST')
GAUSS CAPITAL GESTORA DE RECURSOS LTDA	('Gauss', r'GAUSS')
GAVEA GESTAO DE INVESTIMENTOS LTDA	('Gavea', r'GAVEA')
GAVEA INVESTIMENTOS LTDA	('Gavea', r'GAVEA')
GÁVEA JUS I LTDA.	('Gávea', r'GÁVEA')
GBI CAPITAL E GESTÃO DE RECURSOS LTDA	('Gbi', r'GBI')
GBM BRASIL PARTICIPAÇÕES S.A.	('Gbm', r'GBM')
GCB CAPITAL GESTÃO DE RECURSOS DE VALORES MOBILIÁRIOS LTDA.	('Gcb', r'GCB')
GCS GESTÃO DE RECURSOS E INTERMEDIAÇÃO DE NEGÓCIOS EIRELI	('Gcs', r'GCS')
GDFX INVESTIMENTOS GESTÃO DE RECURSOS LTDA.	('Gdfx', r'GDFX')
GENESIS CAPITAL GESTORA DE RECURSOS LTDA.	('Genesis', r'GENESIS')
GENÈVE ASSET MANAGEMENT	('Genève', r'GENÈVE')
GENIAL GESTÃO LTDA.	('Genial', r'GENIAL')
GENIAL INVESTIMENTOS CORRETORA DE VALORES MOBILIÁRIOS S.A.	('Genial', r'GENIAL')
GENOA CAPITAL GESTORA DE RECURSOS LTDA	('Genoa', r'GENOA')
GEO CAPITAL GESTORA DE RECURSOS LTDA	('Geo', r'GEO')
GERAL ASSET MANAGEMENT S/S LTDA	('Geral', r'GERAL')
GERIBÁ INVESTIMENTOS LTDA	('Geribá', r'GERIBÁ')
GERVAL INVESTIMENTOS LTDA.	('Gerval', r'GERVAL')
GESTORA DE RECURSOS ID - GRID LTDA	('Id', r'ID')
GG CAPITAL INVESTIMENTOS LTDA	('Gg', r'GG')
GGP GESTÃO DE PATRIMÔNIO LTDA.	('Ggp', r'GGP')
GHIA GESTÃO DE RECURSOS LTDA.	('Ghia', r'GHIA')
GHT4 ASSET MANAGEMENT LTDA.	('Ght4', r'GHT4')
GHT4 WEALTH MANAGEMENT LTDA.	('Ght4', r'GHT4')
GIAA Investimentos Gestora de Recursos Ltda.	('Giaa', r'GIAA')
GIANT STEPS CAPITAL INVESTIMENTOS	('Giant', r'GIANT')
GIBRA INVEST ASSET MANAGEMENT LTDA.	('Gibra', r'GIBRA')
GL ASSET GESTÃO DE ATIVOS LTDA.	('Gl', r'GL')
GLOBAL ASSET GESTORA DE RECURSOS LTDA	('Global', r'GLOBAL')
GLOBAL CAPITAL 2000 ADMINISTRADORA DE RECURSOS FINANCEIROS S.A.	('Global', r'GLOBAL')
GLOBAL EQUITY ADMINISTRADORA DE RECURSOS S/A	('Global', r'GLOBAL')
GLOBAL GESTÃO E INVESTIMENTOS LTDA	('Global', r'GLOBAL')
GLOBAL X BRASIL GESTORA DE INVESTIMENTOS LTDA.	('Global', r'GLOBAL')
GOLDMAN SACHS ASSET MANAGEMENT BRASIL LTDA.	('Goldman', r'GOLDMAN')
GOLDMAN SACHS DO BRASIL BANCO MULTIPLO S.A.	('Goldman', r'GOLDMAN')
GOLDRING GESTÃO DE RECURSOS LTDA	('Goldring', r'GOLDRING')
GOVERNANCA & GESTAO INVESTIMENTOS LTDA	('Governanca', r'GOVERNANCA')
GOW CAPITAL GESTÃO DE RECURSOS E CONSULTORIA LTDA.	('Gow', r'GOW')
GPM GESTÃO DE RECURSOS LTDA	('Gpm', r'GPM')
GPS PLANEJAMENTO FINANCEIRO LTDA	('Gps', r'GPS')
GRÃO GESTÃO DE RECURSOS LTDA.	('Grão', r'GRÃO')
GRAPHEN INVESTIMENTOS LTDA.	('Graphen', r'GRAPHEN')
GRAPHUS CAPITAL GESTORA DE RECURSOS LTDA	('Graphus', r'GRAPHUS')
GREEN ALTERNATIVE INVESTMENTS ASSET MANAGEMENT S.A.	('Green', r'GREEN')
GREEN CAPITAL INVESTIMENTOS LTDA	('Green', r'GREEN')
GREENWICH GESTÃO DE RECURSOS LTDA	('Greenwich', r'GREENWICH')
GREENWOOD GESTÃO DE RECURSOS LTDA.	('Greenwood', r'GREENWOOD')
GRIF CAPITAL GESTORA DE RECURSOS LTDA.	('Grif', r'GRIF')
GRIMPER CAPITAL GESTORA DE RECURSOS LTDA	('Grimper', r'GRIMPER')
GROU CAPITAL GESTORA DE RECURSOS LTDA.	('Grou', r'GROU')
GRP INVESTIMENTOS LTDA.	('Grp', r'GRP')
GS ALLOCATION INVESTIMENTOS LTDA	('Gs', r'GS')
GTI ADMINISTRAÇÃO DE RECURSOS LTDA	('Gti', r'GTI')
GUARDIAN GESTORA LTDA.	('Guardian', r'GUARDIAN')
GUEPARDO INVESTIMENTOS LTDA	('Guepardo', r'GUEPARDO')
GV ATACAMA CAPITAL LTDA.	('Gv', r'GV')
GWI ASSET MANAGEMENT S.A.	('Gwi', r'GWI')
H2 ASSET MANAGEMENT LTDA	('H2', r'H2')
H2 KAPITAL S.A.	('H2', r'H2')
HAITONG BANCO DE INVESTIMENTO DO BRASIL S.A.	('Haitong', r'HAITONG')
HALTIN CAPITAL GESTAO DE RECURSOS LTDA	('Haltin', r'HALTIN')
HARBOUR CAPITAL ADMINISTRADORA DE CARTEIRAS DE VALORES MOBILIÁRIOS LTDA.	('Harbour', r'HARBOUR')
HARPIA GESTORA DE RECURSOS LTDA.	('Harpia', r'HARPIA')
HASHDEX GESTORA DE RECURSOS LTDA	('Hashdex', r'HASHDEX')
HCAM LTDA.	('Hcam', r'HCAM')
HCO GROUP S.A	('Hco', r'HCO')
HCS GROUP S.A.	('Hcs', r'HCS')
HDI SEGUROS S.A.	('Hdi', r'HDI')
HECTARE CAPITAL GESTORA DE RECURSOS LTDA.	('Hectare', r'HECTARE')
HEDGE ALTERNATIVE INVESTMENTS LTDA	('Hedge', r'HEDGE')
HEDGE INVESTMENTS REAL ESTATE GESTÃO DE RECURSOS LTDA.	('Hedge', r'HEDGE')
HELIUS CAPITAL GESTÃO DE RECURSOS SA.	('Helius', r'HELIUS')
HENKAN PE INVESTIMENTOS LTDA.	('Henkan', r'HENKAN')
HERITAGE CAPITAL PARTNERS GESTÃO DE RECURSOS LTDA.	('Heritage', r'HERITAGE')
HIGH ASSET MANAGEMENT LTDA	('High', r'HIGH')
HIGH GESTÃO E INVESTIMENTOS LTDA	('High', r'HIGH')
HINOV CAPITAL GESTÃO DE RECURSOS LTDA.	('Hinov', r'HINOV')
HIX INVESTIMENTOS LTDA	('Hix', r'HIX')
HOGAN INVESTIMENTOS ADMINISTRAÇÃO DE RECURSOS LTDA	('Hogan', r'HOGAN')
HORIZONTE CAPITAL GESTÃO DE INVESTIMENTOS LTDA.	('Horizonte', r'HORIZONTE')
HORTA INVESTIMENTOS LTDA	('Horta', r'HORTA')
HORTO GESTORA DE RECURSOS LTDA	('Horto', r'HORTO')
HOSSEGOR GESTÃO DE RECURSOS LTDA.	('Hossegor', r'HOSSEGOR')
HRD ADMINISTRACAO DE RECURSOS LTDA	('Hrd', r'HRD')
HSBC VIDA E PREVIDÊNCIA (BRASIL) S.A.	('Hsbc', r'HSBC')
HSI GESTORA DE CRÉDITO PRIVADO LTDA.	('Hsi', r'HSI')
HSI GESTORA DE FUNDOS IMOBILIÁRIOS LTDA.	('Hsi', r'HSI')
HSI GESTORA DE REAL ESTATE PRIVATE EQUITY LTDA.	('Hsi', r'HSI')
HYPERION ASSET MANAGEMENT LTDA.	('Hyperion', r'HYPERION')
IBBRA GESTÃO DE RECURSOS LTDA.	('Ibbra', r'IBBRA')
IBIUNA AÇÕES GESTÃO DE RECURSOS LTDA.	('Ibiuna', r'IBIUNA')
IBIUNA CRÉDITO GESTÃO DE RECURSOS LTDA.	('Ibiuna', r'IBIUNA')
IBIUNA MACRO GESTÃO DE RECURSOS LTDA.	('Ibiuna', r'IBIUNA')
IBIUNA PREVIDÊNCIA GESTÃO DE RECURSOS LTDA.	('Ibiuna', r'IBIUNA')
ICATU GESTÃO PATRIMONIAL LTDA	('Icatu', r'ICATU')
ICATU SEGUROS S/A	('Icatu', r'ICATU')
ICATU VANGUARDA GESTÃO DE RECURSOS LTDA	('Icatu', r'ICATU')
ICON CAPITAL LTDA.	('Icon', r'ICON')
ID EXCLUSIVO GESTORA DE RECURSOS LTDA.	('Id', r'ID')
ID GESTORA E ADMINISTRADORA DE RECURSOS LTDA	('Id', r'ID')
IF ASSET GESTÃO DE RECURSOS LTDA.	('If', r'IF')
IGF WEALTH MANAGEMENT LTDA	('Igf', r'IGF')
IGUANA INVESTIMENTOS LTDA.	('Iguana', r'IGUANA')
IGUATEMI GESTAO DE CARTEIRAS DE VALORES MOBILIARIOS LTDA	('Iguatemi', r'IGUATEMI')
IMPACTO INVESTIMENTOS LTDA	('Impacto', r'IMPACTO')
INCENTIVO INVESTIMENTOS LTDA	('Incentivo', r'INCENTIVO')
INCENTIVO PARTICIPAÇÕES S.A.	('Incentivo', r'INCENTIVO')
INDIANA SEGUROS S/A	('Indiana', r'INDIANA')
INDIE CAPITAL INVESTIMENTOS LTDA.	('Indie', r'INDIE')
INDUSTRIAL DO BRASIL DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA	('Industrial', r'INDUSTRIAL')
INFINITY ASSET MANAGEMENT ADMINISTRAÇÃO DE RECURSOS LTDA	('Infinity', r'INFINITY')
INFRA ASSET MANAGEMENT LTDA	('Infra', r'INFRA')
INFRA GESTORA DE RECURSOS LTDA.	('Infra', r'INFRA')
INTEGRAL INVESTIMENTOS LTDA.	('Integral', r'INTEGRAL')
INTER-AÇÃO ADMINISTRAÇÃO DE RECURSOS LTDA.	('Interação', r'INTERAÇÃO')
INTER ASSET GESTÃO DE RECURSOS LTDA.	('Inter', r'INTER')
INTERCAP INVESTIMENTOS LTDA	('Intercap', r'INTERCAP')
INTERFLOAT HZ AGENTE AUTÔNOMO DE INVESTIMENTO - EIRELI	('Interfloat', r'INTERFLOAT')
INTRA BLACK INVESTIMENTOS GESTÃO DE RECURSOS LTDA	('Intra', r'INTRA')
INTRABANK ASSET MANAGEMENT LTDA.	('Intrabank', r'INTRABANK')
INTRAG DTVM LTDA.	('Intrag', r'Intrag')
INVEST.PRO CONSULTORIA DE INVESTIMENTOS LTDA.	('Investpro', r'INVESTPRO')
INVESTCENTER ASSET MANAGEMENT LTDA.	('Investcenter', r'INVESTCENTER')
INVESTCOOP ASSET MANAGEMENT LTDA.	('Investcoop', r'INVESTCOOP')
INVESTFORT GESTÃO DE INVESTIMENTOS LTDA	('Investfort', r'INVESTFORT')
INVESTIDOR PROFISSIONAL GESTÃO DE RECURSOS LTDA	('Investidor', r'INVESTIDOR')
INVESTMENT ONE PARTNERS GESTÃO DE RECURSOS LTDA.	('Investment', r'INVESTMENT')
INVESTO GESTÃO DE RECURSOS LTDA.	('Investo', r'INVESTO')
INVESTPORT GESTÃO E CONSULTORIA DE INVESTIMENTOS LTDA	('Investport', r'INVESTPORT')
INVISTA GESTORA DE RECURSOS LTDA.	('Invista', r'INVISTA')
INVISTA REAL ESTATE LTDA.	('Invista', r'INVISTA')
INVISTO GESTÃO DE INVESTIMENTOS S/A	('Invisto', r'INVISTO')
INX ADMINISTRADORA E GESTORA DE RECURSOS LTDA	('Inx', r'INX')
IPANEMA GESTORA DE RECURSOS LTDA.	('Ipanema', r'IPANEMA')
IPORANGA INVESTIMENTOS LTDA	('Iporanga', r'IPORANGA')
IRB ASSET MANAGEMENT S.A.	('Irb', r'IRB')
IRIDIUM GESTÃO DE RECURSOS LTDA.	('Iridium', r'IRIDIUM')
IRON CAPITAL GESTÃO DE RECURSOS LTDA.	('Iron', r'IRON')
ITAU CORRETORA DE VALORES SA	('Itaú', r'Ita(ú|u)')
ITAÚ INVESTMENT SOLUTIONS S.A.	('Itaú', r'Ita(ú|u)')
ITAU UNIBANCO ASSET MANAGEMENT LTDA.	('Itaú', r'Ita(ú|u)')
ITAU UNIBANCO S.A.	('Itaú', r'Ita(ú|u)')
ITAUBANK ASSET MANAGEMENT LTDA	('Itaú', r'Ita(ú|u)')
ITAUBANK DTVM S/A	('Itaú', r'Ita(ú|u)')
ITAÚNA CAPITAL LTDA.	('Itaú', r'Ita(ú|u)')
IVI CAPITAL MANAGEMENT LTDA.	('Ivi', r'IVI')
IWÁ GESTÃO DE RECURSOS LTDA.	('Iwá', r'IWÁ')
J. P. MORGAN ADMINISTRADORA DE CARTEIRAS BRASIL LTDA	('J', r'J')
J. P. MORGAN GAVEA GESTAO DE PATRIMONIO LTDA	('Gavea', r'GAVEA')
J. P. TUCHÊ ASSET MANAGEMENT ADMINISTRAÇÃO DE CARTEIRAS LTDA	('J', r'J')
JACARANDÁ CAPITAL E GESTÃO LTDA.	('Jacarandá', r'JACARANDÁ')
JARDIM BOTANICO PARTNERS INVESTIMENTOS LTDA	('Jardim', r'JARDIM')
JATAI GESTÃO DE RECURSOS DE TERCEIROS LTDA.	('Jatai', r'JATAI')
JAZZ CAPITAL LTDA.	('Jazz', r'JAZZ')
JERA CAPITAL GESTAO DE RECURSOS LTDA	('Jera', r'JERA')
JG CAPITAL GESTÃO DE ATIVOS LTDA	('Jg', r'JG')
JGP Equities Gestão de Recursos Ltda.	('Jgp', r'JGP')
JGP GESTÃO DE CREDITO LTDA	('Jgp', r'JGP')
JGP GESTÃO DE RECURSOS LTDA	('Jgp', r'JGP')
JGP REAL ESTATE GESTÃO DE RECURSOS LTDA.	('Jgp', r'JGP')
JHSF CAPITAL LTDA.	('Jhsf', r'JHSF')
JIVE ASSET GESTÃO DE RECURSOS LTDA	('Jive', r'JIVE')
JIVE HIGH YIELD GESTÃO DE RECURSOS LTDA.	('Jive', r'JIVE')
JIVE INVESTMENTS GESTÃO DE RECURSOS E CONSULTORIA S.A.	('Jive', r'JIVE')
JMN GESTÃO DE INVESTIMENTOS LTDA.	('Jmn', r'JMN')
JOULE - GESTAO DE RECURSOS E VALORES MOBILIARIOS LTDA	('Joule', r'JOULE')
JOURNEY CAPITAL ADMINISTRACAO DE RECURSOS LTDA	('Journey', r'JOURNEY')
JPP CAPITAL GESTAO DE RECURSOS LTDA	('Jpp', r'JPP')
JPP GESTAO DE RECURSOS LTDA	('Jpp', r'JPP')
JUBARTE STRATEGIC CAPITAL LTDA.	('Jubarte', r'JUBARTE')
JUGIS CAPITAL GESTÃO DE RECURSOS LTDA.	('Jugis', r'JUGIS')
JUS CAPITAL GESTÃO DE RECURSOS LTDA.	('Jus', r'JUS')
KADIMA GESTAO DE INVESTIMENTOS LTDA	('Kadima', r'KADIMA')
KANASTRA ADMINISTRAÇÃO DE RECURSOS LTDA	('Kanastra', r'KANASTRA')
KAPITALO CICLO GESTORA DE RECURSOS FINANCEIROS LTDA	('Kapitalo', r'KAPITALO')
KAPITALO INVESTIMENTOS LTDA.	('Kapitalo', r'KAPITALO')
KAPITALO NEXO GESTÃO DE RECURSOS LTDA.	('Kapitalo', r'KAPITALO')
KBO CAPITAL GESTAO DE RECURSOS LTDA	('Kbo', r'KBO')
KILIMA GESTÃO DE RECURSOS LTDA.	('Kilima', r'KILIMA')
KINEA INVESTIMENTOS LTDA.	('Kinea', r'Kinea')
KINEA PRIVATE EQUITY INVESTIMENTOS S/A	('Kinea', r'Kinea')
KÍNITRO CAPITAL GESTÃO DE RECURSOS DE TERCEIROS LTDA.	('Kínitro', r'KÍNITRO')
KIRON CAPITAL GESTÃO DE RECURSOS LTDA.	('Kiron', r'KIRON')
KIRTON BANK S.A. - BANCO MÚLTIPLO	('Kirton', r'KIRTON')
KOBOLD GESTORA DE FUNDOS LTDA	('Kobold', r'KOBOLD')
KÖLI CAPITAL GESTORA DE RECURSOS S.A.	('Köli', r'KÖLI')
KOMATU GESTORA DE RECURSOS LTDA	('Komatu', r'KOMATU')
KONDOR ADMINISTRADORA E GESTORA DE RECURSOS FINANCEIROS LTDA.	('Kondor', r'KONDOR')
KOSMOS ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIÁRIOS LTDA	('Kosmos', r'KOSMOS')
KP GESTÃO DE RECURSOS LTDA	('Kp', r'KP')
KPR INVESTIMENTOS S/A	('Kpr', r'KPR')
KRATHUS GESTORA DE ATIVOS H LTDA	('Krathus', r'KRATHUS')
KUARÁ CAPITAL GESTORA DE RECURSOS LTDA.	('Kuará', r'KUARÁ')
KUMB INVEST S.A.	('Kumb', r'KUMB')
KYMAS GESTÃO DE RECURSOS LTDA	('Kymas', r'KYMAS')
KYROS GESTAO DE RECURSOS LTDA	('Kyros', r'KYROS')
L2 ADMINISTRADORA DE RECURSOS LTDA	('L2', r'L2')
LACAN INVESTIMENTOS E PARTICIPACOES LTDA	('Lacan', r'LACAN')
LAD CAPITAL GESTORA DE RECURSOS LTDA	('Lad', r'LAD')
LAECO ASSET MANAGEMENT LTDA	('Laeco', r'LAECO')
LAGOSTA CAPITAL LTDA.	('Lagosta', r'LAGOSTA')
LAKEWOOD GESTÃO DE RECURSOS LTDA.	('Lakewood', r'LAKEWOOD')
LAM - LAGOA ASSET MANAGEMENT ADMINISTRAÇÃO DE TÍTULOS E VALORES LTDA	('Lam', r'LAM')
LANX CAPITAL INVESTIMENTOS LTDA	('Lanx', r'LANX')
LAPB GESTAO DE RECURSOS FINANCEIROS LTDA	('Lapb', r'LAPB')
LAPLACE INVESTIMENTOS E GESTAO DE RECURSOS LTDA	('Laplace', r'LAPLACE')
LARUS GESTORA DE RECURSOS LTDA.	('Larus', r'LARUS')
LASTRO RDV DTVM Ltda	('Lastro', r'LASTRO')
LATACHE GESTÃO DE RECURSOS LTDA	('Latache', r'LATACHE')
LATAM ACCESS INVESTIMENTOS S.A.	('Latam', r'LATAM')
LATINOAMERICANA DE INVESTIMENTOS E CAPITAL LTDA	('Latinoamericana', r'LATINOAMERICANA')
LATITUDE INVESTMENT PARTNERS GESTAO E FINANCAS LTDA	('Latitude', r'LATITUDE')
LAVORO ASSET MANAGEMENT LTDA	('Lavoro', r'LAVORO')
LEBLON EQUITIES GESTAO DE RECURSOS LTDA	('Leblon', r'LEBLON')
LEGACY CAPITAL GESTORA DE RECURSOS LTDA.	('Legacy', r'LEGACY')
LEGADO ASSET GESTÃO DE RECURSOS LTDA.	('Legado', r'LEGADO')
LEGATUS CAPITAL GESTORA DE RECURSOS LTDA	('Legatus', r'LEGATUS')
LEGEND WM GESTÃO DE RECURSOS LTDA.	('Legend', r'LEGEND')
LEME GESTÃO DE RECURSOS LTDA.	('Leme', r'LEME')
LEME INVESTIMENTOS LTDA	('Leme', r'LEME')
LEROSA INVESTIMENTOS LTDA.	('Lerosa', r'LEROSA')
LESTE ADMINISTRAÇÃO DE RECURSOS LTDA.	('Leste', r'LESTE')
LESTE CREDIT GESTAO DE RECURSOS LTDA	('Leste', r'LESTE')
LESTE FINANCIAL SERVICES GESTÃO DE RECURSOS LTDA.	('Leste', r'LESTE')
LEV ASSET MANAGEMENT LTDA.	('Lev', r'LEV')
LEVANTE GESTORA DE RECURSOS LTDA.	('Levante', r'LEVANTE')
LHYNQZ - GESTAO DE RECURSOS LTDA	('Lhynqz', r'LHYNQZ')
LIBERTAS ASSET S/A	('Libertas', r'LIBERTAS')
LIBERTY PAULISTA SEGUROS S.A.	('Liberty', r'LIBERTY')
LIFETIME GESTORA DE RECURSOS LTDA.	('Lifetime', r'LIFETIME')
LIFT CAPITAL GESTORA DE RECURSOS LTDA.	('Lift', r'LIFT')
LINKER INVESTIMENTOS LTDA.	('Linker', r'LINKER')
LIONS TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Lions', r'LIONS')
LIS CAPITAL - ADMINISTRADORA E GESTORA DE RECURSOS FINANCEIROS LTDA.	('Lis', r'LIS')
LITUS GESTAO DE RECURSOS LTDA	('Litus', r'LITUS')
LIZAR ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIARIOS LTDA	('Lizar', r'LIZAR')
LOGOS GESTÃO DE RECURSOS LTDA.	('Logos', r'LOGOS')
LOIN INVESTMENTS ADMINISTRADORA DE CARTEIRAS LTDA.	('Loin', r'LOIN')
LONGVIEW CAPITAL LTDA.	('Longview', r'LONGVIEW')
LORINVEST GESTÃO DE RECURSOS LTDA	('Lorinvest', r'LORINVEST')
LOTE 45 PARTICIPAÇÕES LTDA	('Lote', r'LOTE')
LUME INVESTIMENTOS LTDA.	('Lume', r'LUME')
LUMINA CAPITAL MANAGEMENT LTDA.	('Lumina', r'LUMINA')
LUTECE INVESTIMENTOS GESTÃO REC. LTDA	('Lutece', r'LUTECE')
LUXOR INVESTIMENTOS LTDA	('Luxor', r'LUXOR')
M SQUARE INVESTIMENTOS LTDA	('M', r'M')
M8 PARTNERS GESTORA DE RECURSOS LTDA.	('M8', r'M8')
MACQUARIE EQUITIES BRASIL ADMINISTRAÇÃO DE FUNDOS E PARTICIPAÇÕES LTDA	('Macquarie', r'MACQUARIE')
MACROINVEST GESTÃO DE RECURSOS LTDA	('Macroinvest', r'MACROINVEST')
MAGNETIS GESTORA DE RECURSOS LTDA	('Magnetis', r'MAGNETIS')
MAGNUS CAPITAL GESTORA DE RECURSOS LTDA.	('Magnus', r'MAGNUS')
MAINU INVESTIMENTOS LTDA.	('Mainu', r'MAINU')
MAJORA CAPITAL GESTÃO DE RECURSOS	('Majora', r'MAJORA')
MAKALU GESTORA DE RECURSOS LTDA.	('Makalu', r'MAKALU')
MAKENA GESTORA DE RECURSOS LTDA.	('Makena', r'MAKENA')
MAM ASSET MANAGEMENT GESTORA DE RECURSOS LTDA.	('Mam', r'MAM')
MANATÍ CAPITAL MANAGEMENT LTDA.	('Manatí', r'MANATÍ')
MANDARIM CSTB INVESTIMENTOS GESTORA DE RECURSOS LTDA	('Mandarim', r'MANDARIM')
MANDARIM GESTÃO DE ATIVOS LTDA	('Mandarim', r'MANDARIM')
MANTARO CAPITAL LTDA	('Mantaro', r'MANTARO')
MAPFRE INVESTIMENTOS LTDA.	('Mapfre', r'MAPFRE')
MAPFRE PREVIDÊNCIA S.A.	('Mapfre', r'MAPFRE')
MAR ASSET MANAGEMENT GESTORA DE RECURSOS LTDA.	('Mar', r'MAR')
MAR CAPITAL GESTAO DE RECURSOS LTDA	('Mar', r'MAR')
MARAÚ GESTÃO DE RECURSOS LTDA.	('Maraú', r'MARAÚ')
MARDI CAPITAL A.G.R. LTDA	('Mardi', r'MARDI')
MARR GESTÃO DE RECURSOS LTDA.	('Marr', r'MARR')
MASSA FALIDA DA CRUZEIRO DO SUL S.A. DTVM	('Massa', r'MASSA')
MASSA FALIDA DA GRADUAL CORRETORA DE CÂMBIO, TÍTULOS E VALORES MOBILIÁRIOS S. A.	('Massa', r'MASSA')
MASSA FALIDA DA UM INVESTIMENTOS SA CTVM SOB ADM. JUD. DE SALOMÃO, KAIUCA & ABRAHÃO SOC. DE ADV.	('Massa', r'MASSA')
MASSA FALIDA DE DIFERENCIAL CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S/A	('Massa', r'MASSA')
MATRIZ CAPITAL ASSET MANAGEMENT LTDA.	('Matriz', r'MATRIZ')
MAUÁ CAPITAL INVESTIMENTOS ALTERNATIVOS LTDA	('Mauá', r'MAUÁ')
MAUÁ CAPITAL REAL ESTATE LTDA	('Mauá', r'MAUÁ')
MAUD CAPITAL GESTORA DE ATIVOS LTDA.	('Maud', r'MAUD')
MÁXIMA ASSET MANAGEMENT LTDA	('Máxima', r'MÁXIMA')
MAXIPLAN LTDA	('Maxiplan', r'MAXIPLAN')
MAZA INVESTIMENTOS E CORRETORA DE SEGUROS LTDA.	('Maza', r'MAZA')
MB ASSETS ADMINISTRAÇÃO DE VALORES MOBILIÁRIOS LTDA	('Mb', r'MB')
MDI ASSOCIADOS CONSULTORIA E ASSESSORIA EM RECURSOS FINANCEIROS LTDA	('Mdi', r'MDI')
MEDICI GESTÃO DE RECURSOS LTDA.	('Medici', r'MEDICI')
MEL GESTORA DE RECURSOS S.A.	('Mel', r'MEL')
MELPAR INVEST LTDA.	('Melpar', r'MELPAR')
MENESTYS GESTORA DE RECURSOS LTDA	('Menestys', r'MENESTYS')
MERAKI CAPITAL GESTÃO DE RECURSOS LTDA.	('Meraki', r'MERAKI')
MERCAANTE INVESTIMENTOS LTDA	('Mercaante', r'MERCAANTE')
MERCANTIL DO BRASIL CORRETORA S.A. - CAMBIO, TITULOS E VALORES MOBILIARIOS	('Mercantil', r'MERCANTIL')
MERCANTIL DO BRASIL DISTRIBUIDORA S/A TÍTULOS E VALORES MOBILIÁRIOS	('Mercantil', r'MERCANTIL')
MERCATTO GESTAO DE RECURSOS LTDA	('Mercatto', r'MERCATTO')
MERCURIUS GESTORA DE RECURSOS LTDA.	('Mercurius', r'MERCURIUS')
MERCURY GESTÃO DE RECURSOS LTDA.	('Mercury', r'MERCURY')
MÉRITO INVESTIMENTOS SA	('Mérito', r'MÉRITO')
MERRILL LYNCH S/A CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS	('Merrill', r'MERRILL')
META ASSET MANAGEMENT LTDA.	('Meta', r'META')
METRIX INVESTIMENTOS LTDA.	('Metrix', r'METRIX')
Mezzus Capital LTDA.	('Mezzus', r'MEZZUS')
MGN INVESTIMENTOS LTDA.	('Mgn', r'MGN')
MHFT INVESTIMENTOS S.A.	('Mhft', r'MHFT')
MILÊNIO CAPITAL GESTÃO DE INVESTIMENTOS LTDA.	('Milênio', r'MILÊNIO')
MILES CAPITAL LTDA.	('Miles', r'MILES')
MILESTONES ADMINISTRADORA DE RECURSOS LTDA.	('Milestones', r'MILESTONES')
MINT GUADUA CAPITAL GESTORA DE RECURSOS LTDA	('Mint', r'MINT')
MIRABAUD INVESTIMENTOS LTDA.	('Mirabaud', r'MIRABAUD')
MIRANTE INVESTIMENTOS LTDA. ME	('Mirante', r'MIRANTE')
MISSION CO. LTDA.	('Mission', r'MISSION')
MMJ MATURITY ADMINISTRACAO DE CARTEIRA S/C LTDA	('Mmj', r'MMJ')
MMZR GESTORA DE RECURSOS LTDA.	('Mmzr', r'MMZR')
MOAT CAPITAL GESTÃO DE RECURSOS LTDA	('Moat', r'MOAT')
MOAT CAPITAL INVESTIMENTOS GESTORA DE RECURSOS LTDA.	('Moat', r'MOAT')
MÖBIUS CAPITAL GESTÃO DE RECURSOS LTDA.	('Möbius', r'MÖBIUS')
MODAL DTVM LTDA	('Modal', r'MODAL')
MÓDULO CAPITAL GESTÃO DE RECURSOS LTDA	('Módulo', r'MÓDULO')
MOKA GESTORA DE RECURSOS DE TERCEIROS LTDA.	('Moka', r'MOKA')
MONETIZA INVESTIMENTOS LTDA.	('Monetiza', r'MONETIZA')
MONGERAL AEGON GESTÃO DE FUNDOS IMOBILIÁRIOS LTDA.	('Mongeral', r'MONGERAL')
MONGERAL AEGON INVESTIMENTOS LTDA	('Mongeral', r'MONGERAL')
MONGERAL AEGON RENDA VARIÁVEL LTDA.	('Mongeral', r'MONGERAL')
MONGERAL AEGON SEGUROS E PREVIDÊNCIA S.A.	('Mongeral', r'MONGERAL')
MONT CAPITAL GESTÃO E ADMINISTRAÇÃO DE RECURSOS S.A.	('Mont', r'MONT')
MONTE AZUL GESTÃO DE RECURSOS LTDA.	('Monte', r'MONTE')
MONTE BRAVO CORRETORA DE TITULOS E VALORES MOBILIARIOS S/A	('Monte', r'MONTE')
MONTE BRAVO GESTÃO DE RECURSOS LTDA.	('Monte', r'MONTE')
MONTE BRAVO GESTÃO PATRIMONIAL LTDA.	('Monte', r'MONTE')
MONTE CAPITAL MANAGEMENT GESTORA DE RECURSOS S.A.	('Monte', r'MONTE')
MORADA CAPITAL LTDA.	('Morada', r'MORADA')
MORALL CAPITAL ADMINISTRAÇÃO DE RECURSOS	('Morall', r'MORALL')
MORE INVEST GESTORA DE RECURSOS LTDA.	('More', r'MORE')
MORGAN STANLEY ADMINISTRADORA DE CARTEIRAS S.A.	('Morgan', r'MORGAN')
MOS GESTÃO DE INVESTIMENTOS LTDA	('Mos', r'MOS')
MPL GESTÃO DE RECURSOS LTDA.	('Mpl', r'MPL')
MRB CAPITAL GESTORA DE RECURSOS LTDA.	('Mrb', r'MRB')
MULTINVEST CAPITAL ADMINISTRADORA DE RECURSOS LTDA	('Multinvest', r'MULTINVEST')
MULTIPLICA CAPITAL ASSET MANAGEMENT LTDA.	('Multiplica', r'MULTIPLICA')
MULTIPLICA WEALTH MANAGEMENT LTDA.	('Multiplica', r'MULTIPLICA')
MUNGER INVESTIMENTOS GESTORA DE RECURSOS LTDA.	('Munger', r'MUNGER')
MURALHA CAPITAL ASSET MANAGEMENT LTDA	('Muralha', r'MURALHA')
MURANO INVESTIMENTOS GESTÃO DE RECURSOS LTDA	('Murano', r'MURANO')
MVM CONSULTORES ASSOC.S/C LTDA	('Mvm', r'MVM')
NAIA CAPITAL GESTÃO E CONSULTORIA FINANCEIRA LTDA.	('Naia', r'NAIA')
NAMAN CAPITAL LTDA.	('Naman', r'NAMAN')
NASCENTE INVESTIMENTOS ADMINISTRAÇÃO DE VALORES MOBILIÁRIOS S.A.	('Nascente', r'NASCENTE')
NAVI ALLOCATION - ADMINISTRADORA E GESTORA DE RECURSOS FINANCEIROS LTDA.	('Navi', r'NAVI')
NAVI CAPITAL ADMINISTRADORA E GESTORA DE RECURSOS FINANCEIROS LTDA.	('Navi', r'NAVI')
NAVI REAL ESTATE SELECTION - ADMINISTRADORA E GESTORA DE RECURSOS FINANCEIROS LTDA.	('Navi', r'NAVI')
NAVI YIELD - ADMINISTRADORA E GESTORA DE RECURSOS FINANCEIROS LTDA.	('Navi', r'NAVI')
NEBRASKA CAPITAL GESTÃO DE RECURSOS LTDA	('Nebraska', r'NEBRASKA')
NECTON INVESTIMENTOS S.A. CORRETORA DE VALORES MOBILIÁRIOS E COMMODITIES	('Necton', r'NECTON')
NEO EQUITIES GESTÃO DE RECURSOS LTDA.	('Neo', r'NEO')
NEO MULTIMERCADO GESTÃO DE RECURSOS LTDA	('Neo', r'NEO')
NEO PRIVATE EQUITY GESTAO DE RECURSOS LTDA	('Neo', r'NEO')
NEON CORRETORA DE TITULOS E VALORES MOBILIARIOS S.A.	('Neon', r'NEON')
NEON INVESTIMENTOS LTDA	('Neon', r'NEON')
NERO CAPITAL GESTÃO DE RECURSOS LTDA.	('Nero', r'NERO')
NEST INTERNATIONAL ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIÁRIOS LTDA	('Nest', r'NEST')
NEST INVESTIMENTOS LTDA	('Nest', r'NEST')
NETZ ASSET GESTÃO DE RECURSOS LTDA	('Netz', r'NETZ')
NEU ADMINISTRADORA DE RECURSOS DE TERCEIROS LTDA	('Neu', r'NEU')
NEULER CAPITAL GESTÃO DE RECURSOS LTDA	('Neuler', r'NEULER')
NEVES ASSET MANAGEMENT LTDA.	('Neves', r'NEVES')
NEWAVE GESTÃO DE RECURSOS LTDA.	('Newave', r'NEWAVE')
NEWFOUNDLAND IRON GESTORA DE RECURSOS LTDA.	('Newfoundland', r'NEWFOUNDLAND')
NEWFOUNDLAND MALIBU GESTORA DE RECURSOS LTDA.	('Newfoundland', r'NEWFOUNDLAND')
NEWPORT REAL ESTATE GESTORA DE RECURSOS LTDA	('Newport', r'NEWPORT')
NEX GESTÃO DE RECURSOS LTDA.	('Nex', r'NEX')
NEXA FINANCE LTDA.	('Nexa', r'NEXA')
NEXTEP INVESTIMENTOS LTDA.	('Nextep', r'NEXTEP')
NIKOS GESTAO DE RECURSOS LTDA.	('Nikos', r'NIKOS')
NITOR ADMINISTRACAO DE RECURSOS LTDA	('Nitor', r'NITOR')
NIX ASSET MANAGEMENT LTDA	('Nix', r'NIX')
NOAD GESTAO DE RECURSOS LTDA	('Noad', r'NOAD')
NOBEL GESTAO DE RECURSOS LTDA	('Nobel', r'NOBEL')
NOOR ADMINISTRADORA DE RECURSOS LTDA	('Noor', r'NOOR')
NORD GESTORA DE RECURSOS LTDA.	('Nord', r'NORD')
NORTE ASSET MANAGEMENT GESTÃO DE RECURSOS S.A.	('Norte', r'NORTE')
NORTH SEA GESTORA DE RECURSOS LTDA.	('North', r'NORTH')
NORTHWEST GESTÃO DE FUNDOS E INVESTIMENTOS LTDA.	('Northwest', r'NORTHWEST')
NOVA FUTURA GESTORA DE RECURSOS LTDA.	('Nova', r'NOVA')
NOVA MILANO INVESTIMENTOS LTDA	('Nova', r'NOVA')
NOVA OCCAM BRASIL GESTAO DE RECURSOS LTDA.	('Nova', r'NOVA')
NOVA S.R.M. ADMINISTRAÇÃO DE RECURSOS E FINANÇAS S/A	('Nova', r'NOVA')
NOVUS CAPITAL GESTORA DE RECURSOS LTDA.	('Novus', r'NOVUS')
NOVUS CREDITO GESTAO DE RECURSOS LTDA	('Novus', r'NOVUS')
NP ADMINISTRAÇÃO DE RECURSOS LTDA	('Np', r'NP')
NU ASSET MANAGEMENT LTDA	('Nu', r'NU')
Nu Investimentos S.A. - Corretora de Títulos e Valores Mobiliários	('Nu', r'NU')
NUCLEO CAPITAL LTDA	('Nucleo', r'NUCLEO')
NUMBER ASSET BRASIL LTDA.	('Number', r'NUMBER')
NUREA CAPITAL GESTORA DE RECURSOS LTDA	('Nurea', r'NUREA')
NW3 CAPITAL GESTÃO DE RECURSOS LTDA.	('Nw3', r'NW3')
O3 GESTÃO DE RECURSOS LTDA	('O3', r'O3')
OAK INVESTIMENTOS LTDA	('Oak', r'OAK')
OASIS VENTURES LTDA.	('Oasis', r'OASIS')
OBB CAPITAL ASSET MANAGEMENT LTDA.	('Obb', r'OBB')
OBY CAPITAL GESTORA DE RECURSOS LTDA.	('Oby', r'OBY')
OBY EQUITIES GESTORA DE RECURSOS LTDA.	('Oby', r'OBY')
OCCAM BRASIL GESTÃO DE RECURSOS LTDA.	('Occam', r'OCCAM')
OCEANA INVESTIMENTOS ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIÁRIOS LTDA	('Oceana', r'OCEANA')
OCTANTE GESTÃO DE RECURSOS LTDA	('Octante', r'OCTANTE')
OIKOS GESTÃO DE RECURSOS LTDA.	('Oikos', r'OIKOS')
OKEAN INVEST LTDA.	('Okean', r'OKEAN')
OLIVEIRA TRUST DTVM S.A.	('Oliveira', r'OLIVEIRA')
OLIVEIRA TRUST SERVICER S/A	('Oliveira', r'OLIVEIRA')
ONE WEALTH MANAGEMENT GESTORA DE RECURSOS LTDA	('One', r'ONE')
ONYX EQUITY MANAGEMENT GESTORA DE INVESTIMENTOS LTDA	('Onyx', r'ONYX')
ONZE GESTORA DE INVESTIMENTOS LTDA.	('Onze', r'ONZE')
OP CONSULTORIA E GESTÃO PATRIMONIAL LTDA.	('Op', r'OP')
OPEN CAPITAL GESTÃO DE ATIVOS LTDA.	('Open', r'OPEN')
OPPORTUNITY ASSET ADMINISTRADORA DE RECURSOS DE TERCEIROS LTDA	('Opportunity', r'Opportunity')
OPPORTUNITY AUSTER WEALTH MANAGEMENT LTDA.	('Opportunity', r'Opportunity')
OPPORTUNITY EQUITY PARTNERS GESTORA DE RECURSOS LTDA	('Opportunity', r'Opportunity')
OPPORTUNITY GESTÃO DE INVESTIMENTOS E RECURSOS LTDA	('Opportunity', r'Opportunity')
OPPORTUNITY GESTORA DE RECURSOS LTDA	('Opportunity', r'Opportunity')
OPPORTUNITY HDF ADMINISTRADORA DE RECURSOS LTDA.	('Opportunity', r'Opportunity')
OPPORTUNITY LOGICA GESTÃO DE RECURSOS LTDA	('Opportunity', r'Opportunity')
OPPORTUNITY PRIVATE EQUITY GESTORA DE RECURSOS LTDA	('Opportunity', r'Opportunity')
OPTIMUM CAPITAL GESTORA DE RECURSOS LTDA.	('Optimum', r'OPTIMUM')
OPUS GESTÃO DE RECURSOS LTDA	('Opus', r'OPUS')
ÓRAMA DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.	('Órama', r'ÓRAMA')
ÓRAMA GESTÃO DE RECURSOS LTDA	('Órama', r'ÓRAMA')
OREY FINANCIAL BRASIL CAPITAL MARKETS LTDA	('Orey', r'OREY')
ORGANON CAPITAL GESTÃO DE INVESTIMENTOS LTDA.	('Organon', r'ORGANON')
ORI CAPITAL LTDA	('Ori', r'ORI')
ORIGINAL ASSET MANAGEMENT LTDA	('Original', r'ORIGINAL')
ORIZ ASSET MANAGEMENT LTDA.	('Oriz', r'ORIZ')
ORLA EMPREENDIMENTOS S.A.	('Orla', r'ORLA')
ORRAM GESTÃO DE RECURSOS LTDA.	('Orram', r'ORRAM')
OSLO CAPITAL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A.	('Oslo', r'OSLO')
OURO PRETO GESTÃO DE RECURSOS S.A.	('Ouro', r'OURO')
OWN Partners Gestão de Recursos Ltda.	('Own', r'OWN')
PACIFICO GESTÃO DE RECURSOS LTDA	('Pacifico', r'PACIFICO')
PANDHORA INVESTIMENTOS LTDA.	('Pandhora', r'PANDHORA')
PANGEA CAPITAL GESTÃO DE RECURSOS LTDA.	('Pangea', r'PANGEA')
Paragon Capital Ltda.	('Paragon', r'PARAGON')
PARAGUAÇU INVESTIMENTOS LTDA	('Paraguaçu', r'PARAGUAÇU')
PARAMIS BR INVESTIMENTOS LTDA	('Paramis', r'PARAMIS')
Parcitas Ações Gestão de Investimentos Ltda.	('Parcitas', r'PARCITAS')
PARCITAS MACRO GESTÃO DE INVESTIMENTOS LTDA	('Parcitas', r'PARCITAS')
PATAGÔNIA CAPITAL GESTORA DE RECURSOS LTDA.	('Patagônia', r'PATAGÔNIA')
PATRIA HEDGE FUNDS ADMINISTRACAO DE RECURSOS LTDA	('Pátria', r'P(á|a)tria')
PÁTRIA INVESTIMENTOS LTDA	('Pátria', r'P(á|a)tria')
PÁTRIA INVESTIMENTOS LTDA.	('Pátria', r'P(á|a)tria')
PÁTRIA VBI REAL ESTATE GESTÃO DE CARTEIRAS SA	('Pátria', r'P(á|a)tria')
PÁTRIA VBI SECURITIES LTDA.	('Pátria', r'P(á|a)tria')
PATRIMONIAL GESTÃO DE RECURSOS LTDA.	('Patrimonial', r'PATRIMONIAL')
PAVARINI & ÓPICE GESTÃO DE ATIVOS LTDA	('Pavarini', r'PAVARINI')
PAX PARTICIPAÇÕES LTDA	('Pax', r'PAX')
PEAK WEALTH ADVISORY GESTORA DE RECURSOS LTDA.	('Peak', r'PEAK')
PEBAY INVESTIMENTOS LTDA.	('Pebay', r'PEBAY')
PENÍNSULA PARTNERS GESTÃO DE INVESTIMENTOS LTDA.	('Península', r'PENÍNSULA')
PENINSULA PATRIMONIAL GESTÃO DE RECURSOS LTDA.	('Peninsula', r'PENINSULA')
PERENNE INVESTIMENTOS LTDA.	('Perenne', r'PERENNE')
PERFIN EQUITIES ADMINISTRAÇÃO DE RECURSOS LTDA.	('Perfin', r'PERFIN')
PERFIN INFRA ADMINISTRAÇÃO DE RECURSOS	('Perfin', r'PERFIN')
PERFIN WEALTH MANAGEMENT LTDA.	('Perfin', r'PERFIN')
PERIMETER ADMINISTRACAO DE RECURSOS LTDA	('Perimeter', r'PERIMETER')
PERSEVERA GESTÃO DE RECURSOS LTDA.	('Persevera', r'PERSEVERA')
PERSONAL LR ASS.FIN.GEST.REC.L	('Personal', r'PERSONAL')
PETRA ASSET GESTÃO DE INVESTIMENTOS LTDA	('Petra', r'PETRA')
PETRA CAPITAL GESTÃO DE INVESTIMENTOS LTDA	('Petra', r'PETRA')
PHI GLOBAL GESTAO DE RECURSOS LTDA.	('Phi', r'PHI')
PHRONESIS INVESTIMENTOS LTDA.	('Phronesis', r'PHRONESIS')
PHYNANCE CIENCIA E TECNOLOGIA EM INVESTIMENTOS S.A.	('Phynance', r'PHYNANCE')
PIER GESTÃO DE RECURSOS FINANCEIROS LTDA.	('Pier', r'PIER')
PILOTAGE GESTÃO DE RECURSOS LTDA	('Pilotage', r'PILOTAGE')
PIMCO LATIN AMERICA ADMINISTRADORA DE CARTEIRAS LTDA	('Pimco', r'PIMCO')
PINE INVESTIMENTOS DTVM LTDA	('Pine', r'PINE')
PINNACLE ADMINISTRACAO DE RECURSOS LTDA	('Pinnacle', r'PINNACLE')
PIP INVESTIMENTOS E PARTICIPAÇÕES LTDA.	('Pip', r'PIP')
PIPO CAPITAL GESTÃO DE INVESTIMENTOS LTDA.	('Pipo', r'PIPO')
PLANALTO CAPITAL GESTÃO DE RECURSOS LTDA.	('Planalto', r'PLANALTO')
PLANNER CORRETORA DE VALORES S.A.	('Planner', r'PLANNER')
PLATINA INVESTIMENTOS LTDA	('Platina', r'PLATINA')
PLATINUM CAPITAL GESTÃO DE RECURSOS LTDA.	('Platinum', r'PLATINUM')
PLENI GESTÃO DE RECURSOS LTDA	('Pleni', r'PLENI')
PLURAL GESTÃO DE RECURSOS LTDA	('Plural', r'PLURAL')
PLURAL INVESTIMENTOS GESTÃO DE RECURSOS LTDA.	('Plural', r'PLURAL')
PLURIMAX GESTÃO DE RECURSOS LTDA	('Plurimax', r'PLURIMAX')
PMFO ESPORTES E ENTRETENIMENTO GESTAO DE RECURSOS LTDA	('Pmfo', r'PMFO')
PNBY GESTORA DE RECURSOS LTDA.	('Pnby', r'PNBY')
POLÍGONO CAPITAL LTDA	('Polígono', r'POLÍGONO')
POLO CAPITAL GESTÃO DE RECURSOS LTDA	('Polo', r'POLO')
POLO CAPITAL INTERNACIONAL GESTÃO DE RECURSOS LTDA	('Polo', r'POLO')
POLYFACE INVEST S.A.	('Polyface', r'POLYFACE')
PONTA SUL INVESTIMENTOS LTDA.	('Ponta', r'PONTA')
PORTCAPITAL GESTORA E CONSULTORIA DE RECURSOS LTDA	('Portcapital', r'PORTCAPITAL')
PORTO REAL INVESTIMENTOS LTDA.	('Porto', r'PORTO')
PORTO SEGURO GESTORA DE RECURSOS LTDA.	('Porto', r'PORTO')
PORTO SEGURO INVESTIMENTOS LTDA	('Porto', r'PORTO')
PORTOFINO GESTÃO DE RECURSOS LTDA.	('Portofino', r'PORTOFINO')
PORTOGALLO INVESTIMENTOS LTDA.	('Portogallo', r'PORTOGALLO')
PORTOPAR DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA.	('Portopar', r'PORTOPAR')
POSITIVA INVESTIMENTOS LTDA	('Positiva', r'POSITIVA')
PRADA ADMINISTRADORA DE RECURSOS LTDA.	('Prada', r'PRADA')
PRAGMA GESTÃO DE PATRIMÔNIO LTDA	('Pragma', r'PRAGMA')
PRECE - PREVIDENCIA COMPLEMENTAR	('Prece', r'PRECE')
PRECISION INVESTIMENTOS LTDA	('Precision', r'PRECISION')
PRIME CCV EMPREENDIMENTOS E PARTICIPAÇÕES S.A.	('Prime', r'PRIME')
PRIMUS VENTURES CONSULTORIA E GESTÃO LTDA.	('Primus', r'PRIMUS')
PRINCIPAL ASSET MANAGEMENT LTDA.	('Principal', r'PRINCIPAL')
PRINCIPAL GESTÃO DE INVESTIMENTOS LTDA	('Principal', r'PRINCIPAL')
PRINCIPIA CAPITAL MANAGEMENT ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIARIOS LTDA	('Principia', r'PRINCIPIA')
PRINZ GESTORA DE RECURSOS LTDA.	('Prinz', r'PRINZ')
PRISMA CAPITAL LTDA	('Prisma', r'PRISMA')
PRISMA CAPITAL MARKETS LTDA.	('Prisma', r'PRISMA')
PRISMA OIL&GAS LTDA	('Prisma', r'PRISMA')
PRISMA PREVIDÊNCIA LTDA.	('Prisma', r'PRISMA')
PRISMA PRIVATE EQUITY LTDA.	('Prisma', r'PRISMA')
PRISMAINVEST GESTÃO DE RECURSOS LTDA	('Prismainvest', r'PRISMAINVEST')
PRIVATTO ADMINISTRAÇÃO DE PATRIMÔNIO LTDA	('Privatto', r'PRIVATTO')
PRÓPRIO CAPITAL GESTÃO DE RECURSOS LTDA	('Próprio', r'PRÓPRIO')
PROSPER GESTÃO DE RECURSOS LTDA	('Prosper', r'PROSPER')
PROSPEX GESTÃO DE RECURSOS LTDA.	('Prospex', r'PROSPEX')
PROTEUS INVESTIMENTOS LTDA	('Proteus', r'PROTEUS')
PRUMO CAPITAL GESTORA DE RECURSOS LTDA	('Prumo', r'PRUMO')
QI GESTÃO DE RECURSOS LTDA.	('Qi', r'QI')
QLZ GESTÃO DE RECURSOS FINANCEIROS LTDA	('Qlz', r'QLZ')
QORE DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Qore', r'QORE')
QR ASSET GESTORA DE RECURSOS LTDA.	('Qr', r'QR')
QUADRA GESTÃO DE RECURSOS S.A.	('Quadra', r'QUADRA')
QUADRANTE INVESTIMENTOS LTDA	('Quadrante', r'QUADRANTE')
QUAESTUS GESTÃO DE INVESTIMENTOS LTDA.	('Quaestus', r'QUAESTUS')
QUANTIQUE M3 INVESTMENTS LTDA.	('Quantique', r'QUANTIQUE')
QUANTITAS GESTAO DE RECURSOS LTDA.	('Quantitas', r'QUANTITAS')
QUANTIX ADMINISTRACAO DE RECURSOS LTDA	('Quantix', r'QUANTIX')
QUARTER INVESTIMENTOS ASSET MANAGEMENT LTDA	('Quarter', r'QUARTER')
QUARTZ GESTÃO DE RECURSOS LTDA.	('Quartz', r'QUARTZ')
QUARTZO ASSET LTDA	('Quartzo', r'QUARTZO')
QUARTZO INVEST LTDA.	('Quartzo', r'QUARTZO')
QUATÁ GESTÃO DE RECURSOS LTDA	('Quatá', r'QUATÁ')
QUATRINVEST ADMINISTRADORA DE RECURSOS LTDA	('Quatrinvest', r'QUATRINVEST')
R CAPITAL ASSET MANAGEMENT INVESTIMENTOS S.A.	('R', r'R')
R. I.  ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIÁRIOS LTDA	('R', r'R')
R2C GESTORA DE INVESTIMENTOS LTDA	('R2c', r'R2C')
RADAR GESTORA DE RECURSOS LTDA.	('Radar', r'RADAR')
RAFTER GESTÃO DE INVESTIMENTOS LTDA	('Rafter', r'RAFTER')
RAM GESTORA DE RECURSOS LTDA.	('Ram', r'RAM')
RB ASSET MANAGEMENT LTDA	('Rb', r'RB')
RB INVESTIMENTOS GESTÃO DE RECURSOS LTDA.	('Rb', r'RB')
RBC BRASIL PARTICIPAÇÕES LTDA	('Rbc', r'RBC')
RBR GESTÃO DE RECURSOS LTDA	('Rbr', r'RBR')
RBR INFRA GESTORA DE RECURSOS LTDA.	('Rbr', r'RBR')
RBR PRIVATE EQUITY GESTÃO DE RECURSOS LTDA.	('Rbr', r'RBR')
RCB PORTFÓLIOS LTDA.	('Rcb', r'RCB')
REACH CAPITAL INVESTIMENTOS LTDA.	('Reach', r'REACH')
REAG ASSET MANAGEMENT S.A.	('Reag', r'REAG')
REAG BERKANA INVESTIMENTOS LTDA	('Reag', r'REAG')
REAG HIERON INVESTIMENTOS LTDA.	('Reag', r'REAG')
REAG JUS GESTÃO DE ATIVOS JUDICIAIS LTDA.	('Reag', r'REAG')
REAG LEGAL CLAIMS GESTÃO DE ATIVOS LTDA.	('Reag', r'REAG')
REAG PORTFÓLIO SOLUTIONS LTDA.	('Reag', r'REAG')
REAG SPECIALTY FINANCE LTDA.	('Reag', r'REAG')
REAG TRUST ADMINISTRADORA DE RECURSOS LTDA.	('Reag', r'REAG')
REAG WM GESTORA DE PATRIMÔNIO LTDA.	('Reag', r'REAG')
REAL CAPITAL PARTNERS LTDA	('Real', r'REAL')
REAL GRANDEZA - FUNDAÇÃO DE PREVIDÊNCIA E ASSISTÊNCIA SOCIAL	('Real', r'REAL')
REAL INVESTOR ASSET MANAGEMENT LTDA	('Real', r'REAL')
REALASSETS GESTORA DE ATIVOS LTDA	('Realassets', r'REALASSETS')
REDASSET GESTAO DE RECURSOS LTDA	('Redasset', r'REDASSET')
REDITUS INVESTIMENTOS LTDA	('Reditus', r'REDITUS')
REDWOOD ADMINISTRAÇÃO DE RECURSOS LTDA	('Redwood', r'REDWOOD')
REGIA CAPITAL LTDA	('Regia', r'REGIA')
RELIANCE ASSET MANAGEMENT ADMINISTRAÇÃO DE RECURSOS LTDA	('Reliance', r'RELIANCE')
RENASCENCE INVESTIMENTOS LTDA	('Renascence', r'RENASCENCE')
RENDA ASSET ADMINISTRADORA DE RECURSOS LTDA	('Renda', r'RENDA')
RENOVA GESTORA DE RECURSOS LTDA	('Renova', r'RENOVA')
RENTA GESTAO DE RECURSOS LTDA	('Renta', r'RENTA')
RETA ASSET LTDA.	('Reta', r'RETA')
RETURN GESTÃO DE RECURSOS S.A.	('Return', r'RETURN')
RICO CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.	('Rico', r'RICO')
RIO BRAVO INVESTIMENTOS - DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA	('Rio', r'RIO')
RIO BRAVO INVESTIMENTOS LTDA	('Rio', r'RIO')
RIO DAS PEDRAS ADMINISTRAÇÃO E PARTICIPAÇÕES LTDA	('Rio', r'RIO')
RIO GRANDE SEGUROS E PREVIDÊNCIA S.A.	('Rio', r'RIO')
RIO PERFORMANCE GESTÃO DE RECURSOS LTDA	('Rio', r'RIO')
RIO VERDE ADMINISTRADORA DE VALORES MOBILIÁRIOS LTDA - EPP	('Rio', r'RIO')
RIVIERA GESTORA DE RECURSOS LTDA	('Riviera', r'RIVIERA')
RIZA ALLOCATION GESTORA DE RECURSOS LTDA	('Riza', r'RIZA')
RIZA CRÉDITO ESTRUTURADO GESTORA DE RECURSOS LTDA.	('Riza', r'RIZA')
RIZA GESTORA DE RECURSOS LTDA.	('Riza', r'RIZA')
RIZA LÍQUIDOS GESTORA DE RECURSOS LTDA.	('Riza', r'RIZA')
RIZA WEALTH MANAGEMENT GESTORA DE RECURSOS LTDA.	('Riza', r'RIZA')
RJ GESTÃO DE RECURSOS LTDA	('Rj', r'RJ')
RJI CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Rji', r'RJI')
RJI GESTAO & INVESTIMENTOS LTDA	('Rji', r'RJI')
RJX INVESTIMENTO LTDA	('Rjx', r'RJX')
ROCHA OPÇÕES DE INVESTIMENTOS LTDA.	('Rocha', r'ROCHA')
ROMA ASSET MANAGEMENT LTDA	('Roma', r'ROMA')
ROOT CAPITAL - GESTÃO DE RECURSOS LTDA	('Root', r'ROOT')
RPS CAPITAL ADMINISTRADORA DE RECURSOS LTDA	('Rps', r'RPS')
RTI VERTEX INVESTIMENTOS LTDA.	('Rti', r'RTI')
RUBIK CAPITAL ASSET MANAGEMENT LTDA.	('Rubik', r'RUBIK')
RURAL DTVM SA - EM LIQUIDAÇÃO EXTRAJUDICIAL	('Rural', r'RURAL')
RYO GESTÃO DE RECURSOS LTDA.	('Ryo', r'RYO')
S3 CACEIS BRASIL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A	('S3', r'S3')
SAFARI CAPITAL GESTÃO DE RECURSOS LTDA.	('Safari', r'SAFARI')
SAFRA ASSET CORRETORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.	('Safra', r'Safra')
SAFRA ASSET MANAGEMENT LTDA.	('Safra', r'Safra')
SAFRA DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Safra', r'Safra')
SAFRA GESTÃO DE RECURSOS LTDA	('Safra', r'Safra')
SAG INVESTIMENTOS LTDA.	('Sag', r'SAG')
SAGRES INVESTIMENTOS ADMINISTRAÇÃO DE RECURSOS LTDA	('Sagres', r'SAGRES')
SAKS GESTÃO DE RECURSOS LTDA.	('Saks', r'SAKS')
SAMESIDE CONSULTORIA E GESTÃO LTDA.	('Sameside', r'SAMESIDE')
SAN PIETRO GESTÃO DE RECURSOS LTDA.	('San', r'SAN')
SANTA FE INVESTIMENTOS LTDA	('Santa', r'SANTA')
SANTANDER ASSET MANAGEMENT DTVM LTDA.	('Santander', r'Santander')
SANTANDER BRASIL ASSET MANAGEMENT S.A.	('Santander', r'Santander')
SANTANDER BRASIL GESTÃO DE RECURSOS LTDA	('Santander', r'Santander')
SANTANDER DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A.	('Santander', r'Santander')
SÃO JOÃO GESTORA DE RECURSOS LTDA	('São', r'SÃO')
SÃO PAULO GESTORA DE RECURSOS LTDA	('São', r'SÃO')
SÃO PEDRO CAPITAL INVESTIMENTOS S.A.	('São', r'SÃO')
SÃO RAFAEL SOCIEDADE DE PREVIDÊNCIA PRIVADA	('São', r'SÃO')
SARPEN QUANT INVESTMENT LTDA.	('Sarpen', r'SARPEN')
SASTRE GESTÃO DE PATRIMÔNIO LTDA.	('Sastre', r'SASTRE')
SCALARE CAPITAL LTDA.	('Scalare', r'SCALARE')
SCHRODER INVESTMENT MANAGEMENT BRASIL LTDA.	('Schroder', r'SCHRODER')
SCOTIABANK BRASIL S.A. BANCO MULTIPLO	('Scotiabank', r'SCOTIABANK')
SDA  GESTÃO DE RECURSOS LTDA	('Sda', r'SDA')
SECURITY ADM. DE RECURSOS LTDA	('Security', r'SECURITY')
SEFER INVESTIMENTOS DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Sefer', r'SEFER')
SEIVAL INVESTIMENTOS LTDA.	('Seival', r'SEIVAL')
SENECA ASSET MANAGEMENT LTDA.	('Seneca', r'SENECA')
SERFICOM BRASIL GESTÃO DE RECURSOS LTDA.	('Serficom', r'SERFICOM')
SERPROS FUNDO MULTIPATROCINADO	('Serpros', r'SERPROS')
SET INVESTIMENTOS GESTÃO DE ATIVOS LTDA.	('Set', r'SET')
SETE ASSET MANAGEMENT LTDA.	('Sete', r'SETE')
SETTA GESTÃO E GOVERNANÇA PATRIMONIAL LTDA.	('Setta', r'SETTA')
SEVEN POUNDS ASSET MANAGEMENT LTDA.	('Seven', r'SEVEN')
SEXTANTE INVESTIMENTOS LTDA	('Sextante', r'SEXTANTE')
SFA INVESTIMENTOS LTDA.	('Sfa', r'SFA')
SFI INVESTIMENTOS LTDA	('Sfi', r'SFI')
SH ASSET CAPITAL GESTÃO DE RECURSOS LTDA	('Sh', r'SH')
SHARP CAPITAL GESTORA DE RECURSOS LTDA	('Sharp', r'SHARP')
SHARPEN CAPITAL ADMINISTRADORA DE RECURSOS LTDA.	('Sharpen', r'SHARPEN')
SHIFT CAPITAL GESTÃO DE RECURSOS S.A.	('Shift', r'SHIFT')
SHORE CAPITAL GESTAO DE RECURSOS LTDA	('Shore', r'SHORE')
SICOOB DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS LTDA.	('Sicoob', r'SICOOB')
SIGULER GUFF GESTORA DE INVESTIMENTOS (ASSET MANAGEMENT) BRASIL LTDA	('Siguler', r'SIGULER')
SILVERADO GESTÃO E INVESTIMENTOS LTDA	('Silverado', r'SILVERADO')
SIMÉTRICA CONSULTORIA EM INVESTIMENTOS E PARTICIPAÇÕES LTDA.	('Simétrica', r'SIMÉTRICA')
SIMON CAPITAL GESTÃO DE RECURSOS LTDA.	('Simon', r'SIMON')
SIMPAUL CORRETORA DE CÂMBIO E VALORES MOBILIÁRIOS S.A.	('Simpaul', r'SIMPAUL')
SINAI ASSET BRASIL LTDA.	('Sinai', r'SINAI')
SINGULAR CAPITAL LTDA.	('Singular', r'SINGULAR')
SINGULARE CORRETORA DE TITULOS E VALORES MOBILIARIOS S.A.	('Singulare', r'SINGULARE')
SIP CAPITAL EQUITIES INVESTIMENTOS LTDA	('Sip', r'SIP')
SITA GESTÃO DE RECURSOS LTDA	('Sita', r'SITA')
SKADE CAPITAL GESTÃO DE INVESTMENTOS LTDA.	('Skade', r'SKADE')
SKOPOS ADMINISTRADORA DE RECURSOS LTDA	('Skopos', r'SKOPOS')
SKOPOS INVESTIMENTOS LTDA	('Skopos', r'SKOPOS')
SLW ADMINISTRADORA E GESTORA DE RECURSOS LTDA.	('Slw', r'SLW')
SLW CVC LTDA	('Slw', r'SLW')
SMART AGRO INVESTIMENTOS LTDA.	('Smart', r'SMART')
SMARTQUANT INVESTIMENTOS LTDA.	('Smartquant', r'SMARTQUANT')
SMARTSAVE GESTORA DE RECURSOS S.A.	('Smartsave', r'SMARTSAVE')
SOD CAPITAL LTDA.	('Sod', r'SOD')
SOLANA GESTORA DE RECURSOS LTDA	('Solana', r'SOLANA')
SOLARIS GESTÃO DE RECURSOS LTDA.	('Solaris', r'SOLARIS')
SOLE CAPITAL LTDA.	('Sole', r'SOLE')
SOLIS INVESTIMENTOS LTDA	('Solis', r'SOLIS')
SOLUM GESTORA DE RECURSOS LTDA.	('Solum', r'SOLUM')
SOLUTIONS GESTORA DE RECURSOS LTDA.	('Solutions', r'SOLUTIONS')
SOMA ASSET GESTÃO DE INVESTIMENTOS LTDA	('Soma', r'SOMA')
SOMACRED ASSET LTDA.	('Somacred', r'SOMACRED')
SOMMA INVESTIMENTOS S.A.	('Somma', r'SOMMA')
SOMMA MULTI-FAMILY OFFICE S.A.	('Somma', r'SOMMA')
SONAR SERVIÇOS DE INVESTIMENTO LTDA.	('Sonar', r'SONAR')
SONATA GESTORA DE RECURSOS LTDA	('Sonata', r'SONATA')
SOSU CAPITAL GESTÃO DE INVESTIMENTOS LTDA.	('Sosu', r'SOSU')
SOW CAPITAL GESTAO DE INVESTIMENTOS LTDA.	('Sow', r'SOW')
SPARTA ADMINISTRADORA DE RECURSOS LTDA	('Sparta', r'SPARTA')
SPECTRA INVESTIMENTOS LTDA	('Spectra', r'SPECTRA')
SPINELLI S/A CVMC	('Spinelli', r'SPINELLI')
SPN GESTÃO DE INVESTIMENTOS LTDA	('Spn', r'SPN')
SPOT GESTÃO DE RECURSOS LTDA.	('Spot', r'SPOT')
SPRINGS CAPITAL LTDA.	('Springs', r'SPRINGS')
SPX CRÉDITO GESTÃO DE RECURSOS LTDA.	('Spx', r'SPX')
SPX EQUITIES GESTÃO DE RECURSOS LTDA	('Spx', r'SPX')
SPX GESTÃO DE RECURSOS LTDA	('Spx', r'SPX')
SPX PRIVATE EQUITY GESTÃO DE RECURSOS LTDA.	('Spx', r'SPX')
SPX REAL ESTATE GESTÃO DE RECURSOS LTDA.	('Spx', r'SPX')
SPX SOLUÇÕES DE INVESTIMENTOS LTDA.	('Spx', r'SPX')
SQUADRA INVESTIMENTOS - GESTÃO DE RECURSOS LTDA.	('Squadra', r'SQUADRA')
SQUALO CAPITAL GESTORA DE RECURSOS LTDA	('Squalo', r'SQUALO')
SRM Empírica Gestão de Crédito LTDA	('Srm', r'SRM')
STARBOARD ASSET LTDA.	('Starboard', r'STARBOARD')
STEN GESTÃO PATRIMONIAL LTDA.	('Sten', r'STEN')
STEN MFO GESTÃO DE RECURSOS LTDA	('Sten', r'STEN')
STEPSTONE GESTÃO DE RECURSOS LTDA.	('Stepstone', r'STEPSTONE')
STIMA GESTÃO DE RECURSOS LTDA.	('Stima', r'STIMA')
STK CAPITAL GESTORA DE RECURSOS LTDA.	('Stk', r'STK')
STOCK ASSET MANAGEMENT ADMINISTRACAO E GESTAO DE RECURSOS LTDA	('Stock', r'STOCK')
STONEX INVESTIMENTOS LTDA.	('Stonex', r'STONEX')
STRATA CAPITAL GESTÃO DE RECURSOS LTDA	('Strata', r'STRATA')
STRATEGIC PORTFOLIO ADVISORS - GESTORA DE RECURSOS LTDA	('Strategic', r'STRATEGIC')
STRIVO GESTORA DE RECURSOS S/A.	('Strivo', r'STRIVO')
STRIX CAPITAL GESTÃO DE PATRIMÔNIO LTDA.	('Strix', r'STRIX')
STS GAEA CAPITAL E ASSESSORIA LTDA.	('Sts', r'STS')
STUDIO INVESTIMENTOS ADMINISTRADORA DE RECURSOS LTDA.	('Studio', r'STUDIO')
SUESTE CAPITAL GESTÃO DE RECURSOS LTDA	('Sueste', r'SUESTE')
SUL AMERICA INVESTIMENTOS DTVM S.A.	('Sul', r'SUL')
SUL AMÉRICA INVESTIMENTOS GESTORA DE RECURSOS S.A.	('Sul', r'SUL')
SUL AMÉRICA SEGUROS DE PESSOAS E PREVIDÊNCIA S.A.	('Sul', r'SUL')
SUMAUMA CAPITAL GESTÃO DE RECURSOS S.A.	('Sumauma', r'SUMAUMA')
SUNO GESTORA DE RECURSOS LTDA	('Suno', r'SUNO')
SUPERMARINE ADMINISTRAÇÃO DE CARTEIRAS DE VALORES MOBILIÁRIOS LTDA.	('Supermarine', r'SUPERMARINE')
SUPERVIELLE PARTICIPAÇÕES LTDA	('Supervielle', r'SUPERVIELLE')
SUPREMO GESTÃO DE INVESTIMENTOS LTDA.	('Supremo', r'SUPREMO')
SVN GESTORA DE RECURSOS LTDA	('Svn', r'SVN')
SWM GESTAO DE RECURSOS LTDA.	('Swm', r'SWM')
TABOAÇO, NIECKELE E ASSOCIADOS - GESTAO PATRIMONIAL LTDA	('Taboaço', r'TABOAÇO')
TAG INVESTIMENTOS LTDA	('Tag', r'TAG')
TAGUS INVESTIMENTOS LTDA	('Tagus', r'TAGUS')
TAÍBA INVESTIMENTOS LTDA.	('Taíba', r'TAÍBA')
TARPON BR S.A.	('Tarpon', r'TARPON')
TARPON GESTORA DE RECURSOS LTDA	('Tarpon', r'TARPON')
TARPON INVESTIMENTOS S/A	('Tarpon', r'TARPON')
TARUÁ CAPITAL GESTORA DE RECURSOS LTDA.	('Taruá', r'TARUÁ')
TATICA ASSET MANAGEMENT ADMINISTRACAO DE RECURSOS S/C LTDA	('Tatica', r'TATICA')
TÁVOLA CAPITAL GESTÃO DE RECURSOS LTDA.	('Távola', r'TÁVOLA')
TB CAPITAL GESTÃO DE RECURSOS LTDA.	('Tb', r'TB')
TCX CONSULTORIA EM ATIVOS FINANCEIROS LTDA	('Tcx', r'TCX')
TELLUS INVESTIMENTOS E  CONSULTORIA LTDA.	('Tellus', r'TELLUS')
TELOS FUNDAÇÃO EMBRATEL DE SEGURIDADE SOCIAL	('Telos', r'TELOS')
TEMPO CAPITAL GESTAO DE RECURSOS LTDA	('Tempo', r'TEMPO')
TENAX CAPITAL LTDA.	('Tenax', r'TENAX')
TENDÊNCIA ASSET MANAGEMENT LTDA.	('Tendência', r'TENDÊNCIA')
TENDÊNCIA WEALTH MANAGEMENT LTDA	('Tendência', r'TENDÊNCIA')
TENOR CAPITAL GESTÃO DE PATRIMÔNIO LTDA	('Tenor', r'TENOR')
TERA INVESTIMENTOS LTDA.	('Tera', r'TERA')
TERCON INVESTIMENTOS LTDA	('Tercon', r'TERCON')
TERRA ASSET VENTURES GESTÃO DE RECURSOS LTDA.	('Terra', r'TERRA')
TERRA GESTORA DE RECURSOS LTDA.	('Terra', r'TERRA')
TERRA NOVA GESTÃO E ADMINISTRAÇÃO DE NEGÓCIOS LTDA	('Terra', r'TERRA')
TERRA VISTA GESTORA DE RECURSOS LTDA	('Terra', r'TERRA')
TESE CONSULTORIA FINANCEIRA LTDA.	('Tese', r'TESE')
TESSA CAPITAL - ASSESSORIA EMPRESARIAL LTDA.	('Tessa', r'TESSA')
TF GESTÃO DE RECURSOS LTDA	('Tf', r'TF')
TG CORE ASSET LTDA	('Tg', r'TG')
TITANIUM INVEST GESTÃO DE INVESTIMENTOS LTDA.	('Titanium', r'TITANIUM')
TIVIO CAPITAL DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS S.A.	('Tivio', r'TIVIO')
TMF BRASIL SERVICOS DE ADMINISTRACAO DE FUNDOS LTDA	('Tmf', r'TMF')
TORDESILHAS CAPITAL GESTORA DE RECURSOS LTDA.	('Tordesilhas', r'TORDESILHAS')
TORI 21 CAPITAL LTDA.	('Tori', r'TORI')
TORI 21 CONSULTORIA E INVESTIMENTOS LTDA	('Tori', r'TORI')
TORK CAPITAL GESTÃO DE RECURSOS LTDA	('Tork', r'TORK')
TORO ASSET MANAGEMENT SA	('Toro', r'TORO')
TOTEM INVESTIMENTOS E GESTÃO DE RECURSOS LTDA.	('Totem', r'TOTEM')
TOV GESTÃO DE RECURSOS LTDA.	('Tov', r'TOV')
TOWER THREE RV GESTORA DE RECURSOS LTDA.	('Tower', r'TOWER')
TPE GESTORA DE RECURSOS LTDA.	('Tpe', r'TPE')
TRACK CAPITAL GESTÃO DE RECURSOS LTDA	('Track', r'TRACK')
TRANSFERO GESTORA DE RECURSOS LTDA.	('Transfero', r'TRANSFERO')
TRAVESSIA CAPITAL ASSET MANAGEMENT LTDA	('Travessia', r'TRAVESSIA')
TREK INVESTIMENTOS LTDA	('Trek', r'TREK')
TRENDING GESTÃO DE RECURSOS LTDA	('Trending', r'TRENDING')
TRIESTOR ADMINISTRADORA DE CARTEIRA DE VALORES MOBILIÁRIOS LTDA.	('Triestor', r'TRIESTOR')
TRÍGONO CAPITAL LTDA	('Trígono', r'TRÍGONO')
TRILHA INVESTIMENTOS LTDA	('Trilha', r'TRILHA')
TRIO CAPITAL LTDA	('Trio', r'TRIO')
TRIUS CAPITAL GESTÃO DE INVESTIMENTOS E CONSULTORIA LTDA	('Trius', r'TRIUS')
TRIVELLA INVESTIMENTOS S.A.	('Trivella', r'TRIVELLA')
TROON GESTORA DE RECURSOS LTDA.	('Troon', r'TROON')
TROPICO INVESTIMENTOS E PARTICIPACOES LTDA	('Tropico', r'TROPICO')
TRUSTEE DISTRIBUIDORA DE TITULOS E VALORES MOBILIARIOS LTDA.	('Trustee', r'TRUSTEE')
TRUXT INVESTIMENTOS LTDA.	('Truxt', r'TRUXT')
TRX GESTORA DE RECURSOS LTDA	('Trx', r'TRX')
TRY CAPITAL LTDA.	('Try', r'TRY')
TS GESTÃO E CONSULTORIA IMOBILIÁRIA LTDA.	('Ts', r'TS')
TT INVESTIMENTOS LTDA.	('Tt', r'TT')
TUERI GESTORA DE RECURSOS LTDA.	('Tueri', r'TUERI')
TUPI GESTÃO DE INVESTIMENTOS FINANCEIROS LTDA	('Tupi', r'TUPI')
TURIM 21 INVESTIMENTOS LTDA	('Turim', r'TURIM')
TURIM INVESTIMENTOS SAO PAULO LTDA	('Turim', r'TURIM')
TURMALINA GESTÃO E ADMINISTRAÇÃO DE RECURSOS S.A.	('Turmalina', r'TURMALINA')
TYR GESTÃO DE RECURSOS LTDA.	('Tyr', r'TYR')
TYTON CAPITAL INVESTIMENTOS LTDA.	('Tyton', r'TYTON')
U.V. GESTORA DE ATIVOS FINANCEIROS LTDA	('Uv', r'UV')
UAM - ASSESSORIA E GESTÃO DE INVESTIMENTOS LTDA.	('Uam', r'UAM')
UBS BB Corretora de Câmbio, Títulos e Valores Mobiliários S.A.	('Ubs', r'UBS')
UBS BRASIL ADMINISTRADORA DE VALORES MOBILIÁRIOS LTDA.	('Ubs', r'UBS')
UBS CONSENSO INVESTIMENTOS LTDA.	('Ubs', r'UBS')
UBS PACTUAL ASSET MANAGEMENT FI  S/A	('Ubs', r'UBS')
UBS PACTUAL EQUITY GESTORA DE RECURSOS LTDA	('Ubs', r'UBS')
UBS PACTUAL GESTORA DE RECURSOS LTDA	('Ubs', r'UBS')
UF GESTÃO DE RECURSOS FINANCEIROS LTDA.	('Uf', r'UF')
UJAY CAPITAL INVESTIMENTOS LTDA	('Ujay', r'UJAY')
ULBREX ASSET MANAGEMENT LTDA	('Ulbrex', r'ULBREX')
ULTRA-MAR CAPITAL MULTIESTRATÉGIA GESTORA DE RECURSOS LTDA.	('Ultramar', r'ULTRAMAR')
UM INVEST ASSET MANAGEMENT LTDA.	('Um', r'UM')
UNA CAPITAL LTDA	('Una', r'UNA')
UNIBANCO ASSET MANAGEMENT S/A DTVM	('Unibanco', r'UNIBANCO')
ÚNICA ADMINISTRAÇÃO E GESTÃO DE RECURSOS LTDA.	('Única', r'ÚNICA')
UNIMED SEGURADORA S/A	('Unimed', r'UNIMED')
UNITY CAPITAL GESTORA DE INVESTIMENTOS LTDA	('Unity', r'UNITY')
UPSIDE GESTAO DE RECURSOS S/C LTDA	('Upside', r'UPSIDE')
URCA GESTÃO DE RECURSOS LTDA.	('Urca', r'URCA')
URCA GESTORA DE ATIVOS LTDA.	('Urca', r'URCA')
UTILIUM GESTORA DE RECURSOS LTDA	('Utilium', r'UTILIUM')
V&B GESTÃO DE PATRIMÔNIO LTDA.	('Vb', r'VB')
V2 INVESTIMENTOS LTDA	('V2', r'V2')
V8 CAPITAL GESTÃO DE INVESTIMENTOS LTDA.	('V8', r'V8')
VALORA IMOBILIÁRIO E INFRAESTRUTURA LTDA.	('Valora', r'VALORA')
VALORA RENDA FIXA ESTRUTURADOS LTDA.	('Valora', r'VALORA')
VALORA RENDA FIXA LTDA.	('Valora', r'VALORA')
VANQUISH CAPITAL LTDA.	('Vanquish', r'VANQUISH')
VCAP INVESTIMENTOS LTDA	('Vcap', r'VCAP')
VCM GESTÃO DE CAPITAL LTDA	('Vcm', r'VCM')
VECTIS GESTÃO DE RECURSOS LTDA	('Vectis', r'VECTIS')
VECTOR ADMINISTRAÇÃO DE RECURSOS FINANCEIROS LTDA	('Vector', r'VECTOR')
VEGA ASSET MANAGEMENT LTDA.	('Vega', r'VEGA')
VELA INVESTIMENTOS LTDA.	('Vela', r'VELA')
VELT PARTNERS INVESTIMENTOS LTDA	('Velt', r'VELT')
VÊNETO GESTÃO DE RECURSOS LTDA	('Vêneto', r'VÊNETO')
VENICE RC GESTÃO DE RECURSOS LTDA.	('Venice', r'VENICE')
VENTOR INVESTIMENTOS LTDA.	('Ventor', r'VENTOR')
VENTURESTAR SERVIÇOS FINANCEIROS LTDA	('Venturestar', r'VENTURESTAR')
VERA CRUZ ASSET MANAGEMENT LTDA.	('Vera', r'VERA')
VERDE ASSET MANAGEMENT S.A.	('Verde', r'VERDE')
VEREDAS CAPITAL GESTÃO DE RECURSOS LTDA.	('Veredas', r'VEREDAS')
VERONA GESTORA DE INVESTIMENTOS LTDA.	('Verona', r'VERONA')
VERSA GESTORA DE RECURSOS LTDA.	('Versa', r'VERSA')
VERSAL FINANCE GESTÃO DE RECURSOS LTDA	('Versal', r'VERSAL')
VERSE CAPITAL GESTORA DE RECURSOS LTDA	('Verse', r'VERSE')
VERT GESTORA DE RECURSOS FINANCEIROS LTDA.	('Vert', r'VERT')
VERTI CAPITAL S/A	('Verti', r'VERTI')
VGR GESTÃO DE RECURSOS LTDA.	('Vgr', r'VGR')
VILA RICA CAPITAL GESTORA DE RECURSOS LTDA	('Vila', r'VILA')
VINCI ASSET ALLOCATION LTDA.	('Vinci', r'Vinci')
VINCI CAPITAL GESTORA DE RECURSOS LTDA.	('Vinci', r'Vinci')
VINCI CG GESTORA DE RECURSOS LTDA	('Vinci', r'Vinci')
VINCI EQUITIES GESTORA DE RECURSOS LTDA	('Vinci', r'Vinci')
VINCI GESTORA DE RECURSOS LTDA	('Vinci', r'Vinci')
VINCI MAV GESTORA DE RECURSOS LTDA.	('Vinci', r'Vinci')
VINCI REAL ESTATE GESTORA DE RECURSOS LTDA	('Vinci', r'Vinci')
VINCI SOLUÇÕES DE INVESTIMENTOS LTDA	('Vinci', r'Vinci')
VINCI SPS GESTÃO DE RECURSOS LTDA	('Vinci', r'Vinci')
VINCI VIDA E PREVIDÊNCIA S.A.	('Vinci', r'Vinci')
VINLAND CAPITAL MANAGEMENT CRÉDITO PRIVADO GESTORA DE RECURSOS LTDA.	('Vinland', r'VINLAND')
VINLAND CAPITAL MANAGEMENT GESTORA DE RECURSOS LTDA.	('Vinland', r'VINLAND')
VINLAND CAPITAL MANAGEMENT II GESTORA DE RECURSOS LTDA	('Vinland', r'VINLAND')
VINTAGE INVESTIMENTOS LTDA	('Vintage', r'VINTAGE')
VIRTUS NEXUS GESTORA DE RECURSOS LTDA.	('Virtus', r'VIRTUS')
VISÃO PREV SOCIEDADE DE PREVIDENCIA COMPLEMENTAR	('Visão', r'VISÃO')
VISION BRAZIL GESTÃO DE INVESTIMENTOS E PARTICIPAÇÕES LTDA	('Vision', r'VISION')
VISTA CAPITAL GESTORA DE RECURSOS LTDA	('Vista', r'VISTA')
VITIS GESTÃO DE RECURSOS LTDA.	('Vitis', r'VITIS')
VITÓRIA ASSET MANAGEMENT S.A.	('Vitória', r'VITÓRIA')
VIX ADMINISTRACAO DE RECURSOS LTDA	('Vix', r'VIX')
VL GESTORA DE RECURSOS LTDA	('Vl', r'VL')
VLGI ASSET LTDA.	('Vlgi', r'VLGI')
VOKIN ADMINISTRAÇÃO DE RECURSOS LTDA	('Vokin', r'VOKIN')
VOX CAPITAL GESTÃO DE RECURSOS SA	('Vox', r'VOX')
VP CAPITAL ASSET MANAGEMENT LTDA.	('Vp', r'VP')
VR GESTORA DE RECURSOS LTDA.	('Vr', r'VR')
VX Pavarini Distribuidora de Títulos e Valores Mobiliários Ltda.	('Vx', r'VX')
W-CAPITAL GESTÃO DE INVESTIMENTOS LTDA	('Wcapital', r'WCAPITAL')
WARREN BRASIL GESTAO E ADMINISTRACAO DE RECURSOS & CORRETORA DE SEGUROS LTDA.	('Warren', r'WARREN')
WE CAPITAL INVESTIMENTOS LTDA.	('We', r'WE')
WEALTH HIGH GOVERNANCE ASSET MANAGEMENT LTDA.	('Wealth', r'WEALTH')
WEALTH HIGH GOVERNANCE CAPITAL LTDA.	('Wealth', r'WEALTH')
WESTERN ASSET MANAGEMENT COMPANY LIMITADA	('Western', r'WESTERN')
WISE ASSET MANAGEMENT LTDA	('Wise', r'WISE')
WIT GESTÃO DE RECURSOS LTDA.	('Wit', r'WIT')
WITPAR GESTORA DE RECURSOS LTDA.	('Witpar', r'WITPAR')
WMR CAPITAL GESTORA DE RECURSOS LTDA.	('Wmr', r'WMR')
WNT GESTORA DE RECURSOS LTDA	('Wnt', r'WNT')
WRIGHT CAPITAL GESTAO DE RECURSOS LTDA	('Wright', r'WRIGHT')
XMS S/A	('Xms', r'XMS')
XP ADVISORY GESTAO DE RECURSOS LTDA	('XP', r'XP')
XP ALLOCATION ASSET MANAGEMENT LTDA.	('XP', r'XP')
XP GESTÃO DE RECURSOS LTDA	('XP', r'XP')
XP INVESTIMENTOS CCTVM S.A.	('XP', r'XP')
XP SERVIÇOS FINANCEIROS DTVM Ltda.	('XP', r'XP')
XP VIDA E PREVIDÊNCIA S.A.	('XP', r'XP')
XP VISTA ASSET MANAGEMENT LTDA.	('XP', r'XP')
XVI CAPITAL LTDA.	('Xvi', r'XVI')
YARDS ASSET GESTORA DE RECURSOS LTDA.	('Yards', r'YARDS')
YVY CAPITAL ASSET MANAGEMENT LTDA.	('Yvy', r'YVY')
ZAGROS CAPITAL GESTAO DE RECURSOS S.A	('Zagros', r'ZAGROS')
ZEITGEIST TECH INVESTIMENTOS LTDA	('Zeitgeist', r'ZEITGEIST')
ZENITH ASSET MANAGEMENT LTDA	('Zenith', r'ZENITH')
ZERO CINCO UM CAPITAL GESTÃO DE RECURSOS LTDA	('Zero', r'ZERO')
ZERO CONFLICT WEALTH MANAGEMENT LTDA	('Zero', r'ZERO')
ZEUS CAPITAL GESTÃO DE RECURSOS E INVESTIMENTOS LTDA.	('Zeus', r'ZEUS')
ZION GESTAO DE RECURSOS LTDA	('Zion', r'ZION')
ZSYS ASSET MANAGEMENT LTDA.	('Zsys', r'ZSYS')
ZURICH SANTANDER BRASIL SEGUROS E PREVIDENCIA S.A.	('Zurich', r'ZURICH')
ZURICH VIDA E PREVIDENCIA S.A.	('Zurich', r'ZURICH')

In [ ]:
def aplicar_depara(nome, patterns_list):
    if not isinstance(nome, str):
        return None
        
    # Normalização básica para ajudar no match (opcional, dependendo do regex)
    # nome_upper = nome.upper()
    
    for grupo, pattern in patterns_list:
        # Tratamento para nomes curtos (< 4 chars)
        regex_pattern = pattern
        if len(pattern) < 4:
            # Garante que é uma palavra sozinha: Começo de string ou espaço + pattern + Final de string ou fim de palavra/espaço
            # Usando word boundaries \b é bom, mas para caracteres acentuados ou específicos, o controle manual de espaço é mais seguro conforme pedido.
            # Pedido: "ou sozinho ou com um espaço"
            regex_pattern = r'(?:^|\s)' + pattern + r'(?:$|\s)'
        
        # Busca case insensitive
        if re.search(regex_pattern, nome, re.IGNORECASE):
            return grupo
            
    return None # Ou retornar 'Outros' / o próprio nome se preferir

# Testes rápidos de sanidade
testes = [
    ("Itaú Unibanco", "Itaú"),
    ("Itaú BBA", "Itaú"),
    ("BB Gestão", "BB"),
    ("Prev BB", "BB"),
    ("UBBA", None), # Não deve pegar BB
    ("BBA", None), # Não deve pegar BB (patterns são cases sensiveis? regex no codigo acima é IGNORECASE)
]

print("Validando lógica em casos de teste:")
for entrada, esperado in testes:
    resultado = aplicar_depara(entrada, patterns)
    match_ok = (resultado == esperado) or (esperado is None and resultado is None)
    print(f"'{entrada}' -> {resultado} (Esperado: {esperado}) [{'OK' if match_ok else 'FALHA'}]")

# Aplicação no Dataset

In [ ]:
df_gestores['grupo'] = df_gestores['gestor'].apply(lambda x: aplicar_depara(x, patterns))

# Filtrar ou organizar resultado final
# O usuário pediu colunas: "grupo", "gestor", "tabela"
df_final = df_gestores[['grupo', 'gestor', 'tabela']].copy()

# Mostrar exemplos classificados
print("\nExemplos classificados:")
display(df_final[df_final['grupo'].notnull()].head(10))

# Mostrar exemplos NÃO classificados (para aprimorar os patterns)
print("\nExemplos NÃO classificados:")
display(df_final[df_final['grupo'].isnull()].head(10))

In [13]:
import re
re.findall(r'B.{0,3}SIDE', 'B.SIDE WEALTH MANAGEMENT GESTÃO DE RECURSOS LTDA')

['B.SIDE']